# Space Biofilm Project Main Data Reading File

This file reads the ground and flight data to create a ground-> flight regression model based on the percent coverage of the biofilm. It predicts trajectories per base material to predict what the third day's percent coverage of the biofilm looks like. 

## Background 

The data in the OSD datasets are not spatiotemporally connected, which means they cannot be used to predict future morphology (shape) of biofilms from day to day. We did write a synthetic data code that serves as validation for morphology, so a future dataset could theoretically be tested using that code. Instead, the focus shifted to looking at the percentage of the sample coupon that was covered in a biofilm. 

In [ ]:
# -*- coding: utf-8 -*-
"""
This file does the following, and is the first file that was used to create a baseline method for this file.
- Reads 'GROUND DATA' and 'FLIGHT DATA' from *_SUBMITTED.xlsx
- Cleans headers; consolidates duplicate columns; coerces numerics
- Parses Material AND robust BaseMaterial + DayInt
- (A) Ground→Flight regression (dynamic features)
- (B) Domain-aware regression with BaseMaterial + DayInt + condition
- (C) Trajectories per BaseMaterial; predict Flight Day 3 (Methods A & B)
- Writes CSVs, PNGs, JSON metrics to /content/lsds55_outputs
"""

import re
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
OUTDIR = Path("/content/lsds55_outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:

    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):

    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:

    df = df.copy()
    merged_info = []
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue

        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]

        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
        merged_info.append((name, len(idxs)))
    if merged_info:
        print("Consolidated duplicate columns:", merged_info)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
  
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true) == 0 or len(y_pred) == 0:
        return {"note": "no rows for evaluation"}
    out = {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }
    out["r2"] = float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan
    out["n"] = int(len(y_true))
    return out

xl = pd.ExcelFile(EXCEL_PATH)
have_ground = "GROUND DATA" in xl.sheet_names
have_flight = "FLIGHT DATA" in xl.sheet_names
if not (have_ground or have_flight):
    raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' sheets found.")

dfs = []
if have_ground:
    g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
    g = normalize_headers(g).dropna(how="all")
    g["condition"] = "ground"
    dfs.append(g)

if have_flight:
    f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
    f = normalize_headers(f).dropna(how="all")
    f["condition"] = "flight"
    dfs.append(f)

df_all = pd.concat(dfs, ignore_index=True)
df_all = clean_cols(df_all)

# Consolidate duplicates before coercion
cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)

# Parse Material & 'day' (legacy) + robust BaseMaterial & DayInt (for modeling)
if "Material and Incubation day" not in df_all.columns:
    raise RuntimeError("Missing 'Material and Incubation day' after header normalization/merge.")
df_all[["Material", "day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
df_all[["BaseMaterial", "DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))

# Coerce numerics
df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])

# Keep essentials
df = df_all.dropna(subset=["Material", "day", "BaseMaterial", "DayInt", "condition"]).copy()
df.to_csv(OUTDIR / "cleaned_combined.csv", index=False)

existing_feats = [c for c in EXPECTED_FEATURES if c in df.columns]
missing_feats = [c for c in EXPECTED_FEATURES if c not in df.columns]

# Fallback: all numeric columns except TARGET & day-like fields
if len(existing_feats) < len(EXPECTED_FEATURES):
    numeric_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
    fallback_exclude = {TARGET, "day", "DayInt", "condition_bin"}
    fallback_feats = [c for c in numeric_candidates if c not in fallback_exclude]
    feature_cols = existing_feats + [c for c in fallback_feats if c not in existing_feats]
else:
    feature_cols = existing_feats

# Remove columns that are all-NaN
feature_cols = [c for c in feature_cols if df[c].notna().any()]

summary = {
    "rows_total": int(len(df_all)),
    "rows_used": int(len(df)),
    "conditions": df["condition"].value_counts(dropna=False).to_dict(),
    "materials_legacy": df["Material"].value_counts().to_dict(),
    "base_materials": df["BaseMaterial"].value_counts().to_dict(),
    "days_legacy": df["day"].value_counts().sort_index().to_dict(),
    "days": df["DayInt"].value_counts().sort_index().to_dict(),
    "features_used_for_A": feature_cols,
    "features_missing_from_expected": missing_feats,
}
with open(OUTDIR / "summary.json", "w") as f:
    json.dump(summary, f, indent=2)

metrics = {}
if ("ground" in df["condition"].unique()) and ("flight" in df["condition"].unique()):
    df_g = df[(df["condition"]=="ground") & df[TARGET].notna()]
    df_f = df[(df["condition"]=="flight") & df[TARGET].notna()]
    if not df_g.empty and not df_f.empty and len(feature_cols) > 0:
        X_train = df_g[feature_cols].values
        y_train = df_g[TARGET].values
        X_test  = df_f[feature_cols].values
        y_test  = df_f[TARGET].values

        rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred_f = rf.predict(X_test)

        metrics["g2f_regression"] = met(y_test, y_pred_f)
        metrics["g2f_regression"].update({
            "n_train_ground": int(len(df_g)),
            "n_test_flight": int(len(df_f)),
            "n_features": int(len(feature_cols)),
        })

        pred_df = df_f.copy()
        pred_df["predicted_coverage_g2f"] = y_pred_f
        pred_df.to_csv(OUTDIR / "g2f_regression_predictions.csv", index=False)

        # Pred vs actual
        plt.figure()
        plt.scatter(y_test, y_pred_f)
        plt.xlabel("Actual Flight Coverage")
        plt.ylabel("Predicted Flight Coverage (trained on Ground)")
        plt.title("Ground→Flight coverage regression")
        lo = float(min(y_test.min(), y_pred_f.min()))
        hi = float(max(y_test.max(), y_pred_f.max()))
        plt.plot([lo, hi], [lo, hi], linestyle="--")
        plt.tight_layout()
        plt.savefig(OUTDIR / "g2f_pred_vs_actual.png", dpi=150)
        plt.close()

        # Feature importances
        imp = pd.DataFrame({"feature": feature_cols, "importance": rf.feature_importances_}).sort_values("importance", ascending=False)
        imp.to_csv(OUTDIR / "g2f_feature_importances.csv", index=False)
        if len(feature_cols) > 1:
            plt.figure()
            plt.barh(imp["feature"], imp["importance"])
            plt.xlabel("Importance")
            plt.title("RF Feature Importances (G→F)")
            plt.tight_layout()
            plt.savefig(OUTDIR / "g2f_feature_importances.png", dpi=150)
            plt.close()
    else:
        metrics["g2f_regression"] = {"note": "Insufficient ground/flight rows with target or no usable features."}
else:
    metrics["g2f_regression"] = {"note": "Require both ground and flight; one missing."}

df_b = df[df[TARGET].notna()].copy()
df_b["condition_bin"] = (df_b["condition"]=="flight").astype(int)

# Build numeric features dynamically for (B)
num_candidates = df_b.select_dtypes(include=[np.number]).columns.tolist()
num_candidates = [c for c in num_candidates if c != TARGET]
# Always include DayInt and condition_bin if present; put them at the end for clarity
num_cols = [c for c in num_candidates if c not in ["DayInt","condition_bin"]] + [c for c in ["DayInt","condition_bin"] if c in num_candidates or c in df_b.columns]
num_cols = [c for c in num_cols if c in df_b.columns]
cat_cols = ["BaseMaterial"] if "BaseMaterial" in df_b.columns else []

if not df_b.empty and len(num_cols + cat_cols) > 0:
    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
        ],
        remainder="drop"
    )
    rf_dom = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
    pipe_dom = Pipeline([("pre", pre), ("rf", rf_dom)])
    pipe_dom.fit(df_b[num_cols + cat_cols], df_b[TARGET])

    df_b["pred_domain_model_base"] = pipe_dom.predict(df_b[num_cols + cat_cols])
    df_b.to_csv(OUTDIR / "domain_aware_predictions_base_material.csv", index=False)

    for cond in ["ground","flight"]:
        sub = df_b[df_b["condition"]==cond]
        if not sub.empty:
            metrics[f"domain_model_base_{cond}"] = met(sub[TARGET].values, sub["pred_domain_model_base"].values)
            metrics[f"domain_model_base_{cond}"]["n"] = int(len(sub))
        else:
            metrics[f"domain_model_base_{cond}"] = {"note": "no rows"}
else:
    metrics["domain_model_base"] = {"note": "No rows or no usable features for domain-aware model."}

plots_dir = OUTDIR / "trajectories_base"
plots_dir.mkdir(exist_ok=True)

traj_rows = []
have_flight = "flight" in df["condition"].unique()
materials = sorted(df["BaseMaterial"].dropna().unique())

for mat in materials:
    g_sub = df[(df["condition"]=="ground") & (df["BaseMaterial"]==mat)]
    f_sub = df[(df["condition"]=="flight") & (df["BaseMaterial"]==mat)] if have_flight else pd.DataFrame()

    # Need ≥2 ground days to fit slope
    if g_sub["DayInt"].nunique() < 2:
        continue

    # Linear ground trajectory: coverage ~ DayInt
    Xg = g_sub[["DayInt"]].values
    yg = g_sub[TARGET].values
    lin = LinearRegression().fit(Xg, yg)

    slope = float(lin.coef_[0])
    intercept = float(lin.intercept_)

    # Ground per-day growth rate if Day1 and Day3 exist
    growth_rate = np.nan
    if (g_sub["DayInt"]==1).any() and (g_sub["DayInt"]==3).any():
        cov_d1 = g_sub.loc[g_sub["DayInt"]==1, TARGET].mean()
        cov_d3 = g_sub.loc[g_sub["DayInt"]==3, TARGET].mean()
        growth_rate = (cov_d3 - cov_d1) / 2.0

    # Method A: ground line at day 3
    pred_day3_A = float(lin.predict(np.array([[3]]) )[0])

    # Method B: Flight Day1 + ground growth_rate * 2 (if available)
    pred_day3_B = np.nan
    actual_flight_d3 = np.nan
    if have_flight and not f_sub.empty:
        f_d1 = f_sub[f_sub["DayInt"]==1]
        f_d3 = f_sub[f_sub["DayInt"]==3]
        if not f_d3.empty:
            actual_flight_d3 = float(f_d3[TARGET].mean())
        if not f_d1.empty and not np.isnan(growth_rate):
            pred_day3_B = float(f_d1[TARGET].mean() + 2.0*growth_rate)

    traj_rows.append({
        "BaseMaterial": mat,
        "ground_lin_slope": slope,
        "ground_lin_intercept": intercept,
        "ground_growth_rate_per_day": growth_rate,
        "predicted_flight_day3_methodA_ground_line": pred_day3_A,
        "predicted_flight_day3_methodB_flightD1_plus_ground_rate": pred_day3_B,
        "actual_flight_day3": actual_flight_d3
    })

    # Plot per base material
    plt.figure()
    plt.scatter(g_sub["DayInt"], g_sub[TARGET], label="Ground (actual)")
    xs = np.array([[1],[2],[3]])
    ys = lin.predict(xs)
    plt.plot(xs.flatten(), ys, linestyle="--", label="Ground fit")
    if have_flight and not f_sub.empty:
        plt.scatter(f_sub["DayInt"], f_sub[TARGET], label="Flight (actual)")
        if not np.isnan(pred_day3_B):
            plt.scatter([3], [pred_day3_B], marker="x", label="Pred Flight D3 (Method B)")
        plt.scatter([3], [pred_day3_A], marker="^", label="Pred Flight D3 (Method A)")
    plt.xlabel("Day")
    plt.ylabel("Biofilm surface area coverage (%)")
    plt.title(f"Trajectory (BaseMaterial): {mat}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plots_dir / f"trajectory_{re.sub(r'[^A-Za-z0-9]+','_',mat)}.png", dpi=150)
    plt.close()

traj_df = pd.DataFrame(traj_rows)
traj_df.to_csv(OUTDIR / "trajectory_predictions_base_material.csv", index=False)

# Evaluate trajectory predictions where Flight Day3 exists
def eval_block(df_eval, pred_col):
    if df_eval.empty or pred_col not in df_eval.columns:
        return {"note": "no comparable rows"}
    mask = (~df_eval["actual_flight_day3"].isna()) & (~df_eval[pred_col].isna())
    y_true = df_eval.loc[mask, "actual_flight_day3"].values
    y_pred = df_eval.loc[mask, pred_col].values
    if len(y_true) == 0:
        return {"note": "no comparable rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)) if len(np.unique(y_true))>1 else np.nan,
        "n": int(len(y_true))
    }

metrics["trajectory_base_methodA"] = eval_block(traj_df, "predicted_flight_day3_methodA_ground_line")
metrics["trajectory_base_methodB"] = eval_block(traj_df, "predicted_flight_day3_methodB_flightD1_plus_ground_rate")

# Save metrics + README
with open(OUTDIR / "metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

with open(OUTDIR / "README.txt", "w") as f:
    f.write(
f"""Outputs: {OUTDIR}
"""
    )


Consolidated duplicate columns: [('Biofilm mass (µm^3/µm^2)', 2)]
Done. Outputs in: /content/lsds55_outputs


quadratic (degree-2) trajectory fit for the Ground curves, then using that to predict Flight Day-3

In [ ]:
# -*- coding: utf-8 -*-
"""
numeric only pipeline with two sheet excel for LSDS
"""

import re
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.polynomial.polynomial import polyfit, polyval

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
OUTDIR = Path("/content/lsds55_outputs")
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
  
    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    """Return (Material, day_int) from strings like 'SS316 day1' or 'Silicone  day 3'."""
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):
  
    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:
 
    df = df.copy()
    merged_info = []
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue

        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]

        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
        merged_info.append((name, len(idxs)))
    if merged_info:
        print("Consolidated duplicate columns:", merged_info)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    Coerce listed columns to numeric, dropping extra duplicates if any remain.
    """
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true) == 0 or len(y_pred) == 0:
        return {"note": "no rows for evaluation"}
    out = {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }
    out["r2"] = float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan
    out["n"] = int(len(y_true))
    return out

xl = pd.ExcelFile(EXCEL_PATH)
have_ground = "GROUND DATA" in xl.sheet_names
have_flight = "FLIGHT DATA" in xl.sheet_names
if not (have_ground or have_flight):
    raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' sheets found.")

dfs = []
if have_ground:
    g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
    g = normalize_headers(g).dropna(how="all")
    g["condition"] = "ground"
    dfs.append(g)

if have_flight:
    f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
    f = normalize_headers(f).dropna(how="all")
    f["condition"] = "flight"
    dfs.append(f)

df_all = pd.concat(dfs, ignore_index=True)
df_all = clean_cols(df_all)

# Consolidate duplicates before coercion
cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)

# Parse Material & 'day' (legacy) + robust BaseMaterial & DayInt (for modeling)
if "Material and Incubation day" not in df_all.columns:
    raise RuntimeError("Missing 'Material and Incubation day' after header normalization/merge.")
df_all[["Material", "day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
df_all[["BaseMaterial", "DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))

# Coerce numerics
df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])

# Keep essentials
df = df_all.dropna(subset=["Material", "day", "BaseMaterial", "DayInt", "condition"]).copy()
df.to_csv(OUTDIR / "cleaned_combined.csv", index=False)

existing_feats = [c for c in EXPECTED_FEATURES if c in df.columns]
missing_feats = [c for c in EXPECTED_FEATURES if c not in df.columns]

# Fallback: all numeric columns except TARGET & day-like fields
if len(existing_feats) < len(EXPECTED_FEATURES):
    numeric_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
    fallback_exclude = {TARGET, "day", "DayInt", "condition_bin"}
    fallback_feats = [c for c in numeric_candidates if c not in fallback_exclude]
    feature_cols = existing_feats + [c for c in fallback_feats if c not in existing_feats]
else:
    feature_cols = existing_feats

# Remove columns that are all-NaN
feature_cols = [c for c in feature_cols if df[c].notna().any()]

summary = {
    "rows_total": int(len(df_all)),
    "rows_used": int(len(df)),
    "conditions": df["condition"].value_counts(dropna=False).to_dict(),
    "materials_legacy": df["Material"].value_counts().to_dict(),
    "base_materials": df["BaseMaterial"].value_counts().to_dict(),
    "days_legacy": df["day"].value_counts().sort_index().to_dict(),
    "days": df["DayInt"].value_counts().sort_index().to_dict(),
    "features_used_for_A": feature_cols,
    "features_missing_from_expected": missing_feats,
}
with open(OUTDIR / "summary.json", "w") as f:
    json.dump(summary, f, indent=2)

metrics = {}
if ("ground" in df["condition"].unique()) and ("flight" in df["condition"].unique()):
    df_g = df[(df["condition"]=="ground") & df[TARGET].notna()]
    df_f = df[(df["condition"]=="flight") & df[TARGET].notna()]
    if not df_g.empty and not df_f.empty and len(feature_cols) > 0:
        X_train = df_g[feature_cols].values
        y_train = df_g[TARGET].values
        X_test  = df_f[feature_cols].values
        y_test  = df_f[TARGET].values

        rf = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred_f = rf.predict(X_test)

        metrics["g2f_regression"] = met(y_test, y_pred_f)
        metrics["g2f_regression"].update({
            "n_train_ground": int(len(df_g)),
            "n_test_flight": int(len(df_f)),
            "n_features": int(len(feature_cols)),
        })

        pred_df = df_f.copy()
        pred_df["predicted_coverage_g2f"] = y_pred_f
        pred_df.to_csv(OUTDIR / "g2f_regression_predictions.csv", index=False)

        # Pred vs actual
        plt.figure()
        plt.scatter(y_test, y_pred_f)
        plt.xlabel("Actual Flight Coverage")
        plt.ylabel("Predicted Flight Coverage (trained on Ground)")
        plt.title("Ground→Flight coverage regression")
        lo = float(min(y_test.min(), y_pred_f.min()))
        hi = float(max(y_test.max(), y_pred_f.max()))
        plt.plot([lo, hi], [lo, hi], linestyle="--")
        plt.tight_layout()
        plt.savefig(OUTDIR / "g2f_pred_vs_actual.png", dpi=150)
        plt.close()

        # Feature importances
        imp = pd.DataFrame({"feature": feature_cols, "importance": rf.feature_importances_}).sort_values("importance", ascending=False)
        imp.to_csv(OUTDIR / "g2f_feature_importances.csv", index=False)
        if len(feature_cols) > 1:
            plt.figure()
            plt.barh(imp["feature"], imp["importance"])
            plt.xlabel("Importance")
            plt.title("RF Feature Importances (G→F)")
            plt.tight_layout()
            plt.savefig(OUTDIR / "g2f_feature_importances.png", dpi=150)
            plt.close()
    else:
        metrics["g2f_regression"] = {"note": "Insufficient ground/flight rows with target or no usable features."}
else:
    metrics["g2f_regression"] = {"note": "Require both ground and flight; one missing."}

df_b = df[df[TARGET].notna()].copy()
df_b["condition_bin"] = (df_b["condition"]=="flight").astype(int)

# Build numeric features dynamically for (B)
num_candidates = df_b.select_dtypes(include=[np.number]).columns.tolist()
num_candidates = [c for c in num_candidates if c != TARGET]
# Always include DayInt and condition_bin if present; put them at the end for clarity
num_cols = [c for c in num_candidates if c not in ["DayInt","condition_bin"]] + [c for c in ["DayInt","condition_bin"] if c in df_b.columns]
cat_cols = ["BaseMaterial"] if "BaseMaterial" in df_b.columns else []

if not df_b.empty and len(num_cols + cat_cols) > 0:
    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
        ],
        remainder="drop"
    )
    rf_dom = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
    pipe_dom = Pipeline([("pre", pre), ("rf", rf_dom)])
    pipe_dom.fit(df_b[num_cols + cat_cols], df_b[TARGET])

    df_b["pred_domain_model_base"] = pipe_dom.predict(df_b[num_cols + cat_cols])
    df_b.to_csv(OUTDIR / "domain_aware_predictions_base_material.csv", index=False)

    for cond in ["ground","flight"]:
        sub = df_b[df_b["condition"]==cond]
        if not sub.empty:
            metrics[f"domain_model_base_{cond}"] = met(sub[TARGET].values, sub["pred_domain_model_base"].values)
            metrics[f"domain_model_base_{cond}"]["n"] = int(len(sub))
        else:
            metrics[f"domain_model_base_{cond}"] = {"note": "no rows"}
else:
    metrics["domain_model_base"] = {"note": "No rows or no usable features for domain-aware model."}

plots_dir = OUTDIR / "trajectories_base_quad"
plots_dir.mkdir(exist_ok=True)

traj_rows = []
have_flight = "flight" in df["condition"].unique()
materials = sorted(df["BaseMaterial"].dropna().unique())

for mat in materials:
    g_sub = df[(df["condition"]=="ground") & (df["BaseMaterial"]==mat)]
    f_sub = df[(df["condition"]=="flight") & (df["BaseMaterial"]==mat)] if have_flight else pd.DataFrame()

    # Need ≥2 ground days to fit a curve; deg=2 is stable with 2–3 points
    if g_sub["DayInt"].nunique() < 2:
        continue

    xg = g_sub["DayInt"].values.astype(float)
    yg = g_sub[TARGET].values.astype(float)

    # Quadratic fit: coverage = a0 + a1*x + a2*x^2
    coefs = polyfit(xg, yg, deg=2)  # returns [a0, a1, a2]
    pred_day3_A = float(polyval(3.0, coefs))

    # Ground "per-day" growth rate analogous to linear delta (optional diagnostic)
    growth_rate = np.nan
    if (g_sub["DayInt"]==1).any() and (g_sub["DayInt"]==3).any():
        cov_d1 = g_sub.loc[g_sub["DayInt"]==1, TARGET].mean()
        cov_d3 = g_sub.loc[g_sub["DayInt"]==3, TARGET].mean()
        growth_rate = (cov_d3 - cov_d1) / 2.0

    pred_day3_B = np.nan
    actual_flight_d3 = np.nan
    if have_flight and not f_sub.empty:
        f_d1 = f_sub[f_sub["DayInt"]==1]
        f_d3 = f_sub[f_sub["DayInt"]==3]
        if not f_d3.empty:
            actual_flight_d3 = float(f_d3[TARGET].mean())
        if not f_d1.empty and not np.isnan(growth_rate):
            # Keep Method B identical in spirit to linear version for comparability
            pred_day3_B = float(f_d1[TARGET].mean() + 2.0*growth_rate)

    # Save row
    traj_rows.append({
        "BaseMaterial": mat,
        "quad_a0": float(coefs[0]),
        "quad_a1": float(coefs[1]),
        "quad_a2": float(coefs[2]),
        "ground_growth_rate_per_day_linearized": growth_rate,
        "predicted_flight_day3_methodA_ground_quadratic": pred_day3_A,
        "predicted_flight_day3_methodB_flightD1_plus_ground_rate": pred_day3_B,
        "actual_flight_day3": actual_flight_d3
    })

    # Plot per base material
    xs = np.array([1.0, 2.0, 3.0])
    ys_fit = polyval(xs, coefs)

    plt.figure()
    plt.scatter(xg, yg, label="Ground (actual)")
    plt.plot(xs, ys_fit, linestyle="--", label="Ground quadratic fit")
    if have_flight and not f_sub.empty:
        plt.scatter(f_sub["DayInt"], f_sub[TARGET], label="Flight (actual)")
        if not np.isnan(pred_day3_B):
            plt.scatter([3], [pred_day3_B], marker="x", label="Pred Flight D3 (Method B)")
        plt.scatter([3], [pred_day3_A], marker="^", label="Pred Flight D3 (Method A, quad)")
    plt.xlabel("Day")
    plt.ylabel("Biofilm surface area coverage (%)")
    plt.title(f"Trajectory (Quadratic) — {mat}")
    plt.legend()
    plt.tight_layout()
    plt.savefig(plots_dir / f"trajectory_quad_{re.sub(r'[^A-Za-z0-9]+','_',mat)}.png", dpi=150)
    plt.close()

traj_df = pd.DataFrame(traj_rows)
traj_df.to_csv(OUTDIR / "trajectory_predictions_base_material_quad.csv", index=False)

# Evaluate trajectory predictions where Flight Day3 exists
def eval_block(df_eval, pred_col):
    if df_eval.empty or pred_col not in df_eval.columns:
        return {"note": "no comparable rows"}
    mask = (~df_eval["actual_flight_day3"].isna()) & (~df_eval[pred_col].isna())
    y_true = df_eval.loc[mask, "actual_flight_day3"].values
    y_pred = df_eval.loc[mask, pred_col].values
    if len(y_true) == 0:
        return {"note": "no comparable rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)) if len(np.unique(y_true))>1 else np.nan,
        "n": int(len(y_true))
    }

# Add quadratic metric keys to avoid overwriting linear ones
metrics["trajectory_base_quad_methodA"] = eval_block(traj_df, "predicted_flight_day3_methodA_ground_quadratic")
metrics["trajectory_base_quad_methodB"] = eval_block(traj_df, "predicted_flight_day3_methodB_flightD1_plus_ground_rate")

# Save metrics + README
with open(OUTDIR / "metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

with open(OUTDIR / "README.txt", "w") as f:
    f.write(
f"""Outputs: {OUTDIR}
"""
    )

print("Done. Outputs in:", OUTDIR)


Consolidated duplicate columns: [('Biofilm mass (µm^3/µm^2)', 2)]
Done. Outputs in: /content/lsds55_outputs


testing improvements

In [ ]:
# -*- coding: utf-8 -*-
"""
experiment harness
"""

import re, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold

try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

try:
    import statsmodels.formula.api as smf
    HAS_STATSMODELS = True
except Exception:
    HAS_STATSMODELS = False

EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
ROOT_OUT = Path("/content/lsds55_outputs")
EXP_OUT = ROOT_OUT / "experiments"
EXP_OUT.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):
    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue
        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]
        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true) == 0 or len(y_pred) == 0:
        return {"note": "no rows for evaluation"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan,
        "n": int(len(y_true))
    }

def load_clean_data():
    xl = pd.ExcelFile(EXCEL_PATH)
    dfs = []
    if "GROUND DATA" in xl.sheet_names:
        g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
        g = normalize_headers(g).dropna(how="all")
        g["condition"] = "ground"
        dfs.append(g)
    if "FLIGHT DATA" in xl.sheet_names:
        f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
        f = normalize_headers(f).dropna(how="all")
        f["condition"] = "flight"
        dfs.append(f)
    if not dfs:
        raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' sheets found.")
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = clean_cols(df_all)
    cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
    df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)

    if "Material and Incubation day" not in df_all.columns:
        raise RuntimeError("Missing 'Material and Incubation day'.")
    df_all[["Material", "day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
    df_all[["BaseMaterial", "DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))
    df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])

    df = df_all.dropna(subset=["Material", "day", "BaseMaterial", "DayInt", "condition"]).copy()
    return df

def get_dynamic_features_for_g2f(df, include_dayint=False, add_interactions=False):
    existing = [c for c in EXPECTED_FEATURES if c in df.columns and df[c].notna().any()]
    if len(existing) < len(EXPECTED_FEATURES):
        numeric_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
        exclude = {TARGET, "day", "DayInt", "condition_bin"}
        fallback = [c for c in numeric_candidates if c not in exclude]
        feats = existing + [c for c in fallback if c not in existing]
    else:
        feats = existing
    if add_interactions:
        if ("Biofilm mass (µm^3/µm^2)" in df.columns) and ("roughness coefficient Ra*" in df.columns):
            df["mass_x_roughness"] = df["Biofilm mass (µm^3/µm^2)"] * df["roughness coefficient Ra*"]
            feats.append("mass_x_roughness")
        if ("Biofilm biomass mean thickness (µm)" in df.columns) and ("Biofilm Maximum thickness (µm)" in df.columns):
            df["mean_x_max"] = df["Biofilm biomass mean thickness (µm)"] * df["Biofilm Maximum thickness (µm)"]
            feats.append("mean_x_max")
    if include_dayint and ("DayInt" in df.columns):
        feats.append("DayInt")
    feats = [c for c in feats if c in df.columns and df[c].notna().any()]
    return df, feats

def run_g2f(df, model_ctor, feature_setup_kwargs):
    df_g = df[(df["condition"]=="ground") & df[TARGET].notna()].copy()
    df_f = df[(df["condition"]=="flight") & df[TARGET].notna()].copy()
    if df_g.empty or df_f.empty:
        return {"g2f_regression": {"note": "Insufficient ground/flight rows"}}
    df_aug, feats = get_dynamic_features_for_g2f(df, **feature_setup_kwargs)
    if not feats:
        return {"g2f_regression": {"note": "No usable features"}}
    X_train = df_g[feats].values
    y_train = df_g[TARGET].values
    X_test  = df_f[feats].values
    y_test  = df_f[TARGET].values

    model = model_ctor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    out = met(y_test, y_pred)
    out.update({"n_train_ground": int(len(df_g)), "n_test_flight": int(len(df_f)), "n_features": int(len(feats))})
    return {"g2f_regression": out}

def run_domain_aware(df, model_ctor):
    df_b = df[df[TARGET].notna()].copy()
    if df_b.empty:
        return {"domain_model_base_ground": {"note":"no rows"}, "domain_model_base_flight": {"note":"no rows"}}
    df_b["condition_bin"] = (df_b["condition"]=="flight").astype(int)
    num_candidates = df_b.select_dtypes(include=[np.number]).columns.tolist()
    num_candidates = [c for c in num_candidates if c != TARGET]
    num_cols = [c for c in num_candidates if c not in ["DayInt","condition_bin"]]
    for c in ["DayInt","condition_bin"]:
        if c in df_b.columns and c not in num_cols:
            num_cols.append(c)
    cat_cols = ["BaseMaterial"] if "BaseMaterial" in df_b.columns else []

    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
        ],
        remainder="drop"
    )
    model = model_ctor()
    pipe = Pipeline([("pre", pre), ("model", model)])
    pipe.fit(df_b[num_cols + cat_cols], df_b[TARGET])

    df_b["pred_domain"] = pipe.predict(df_b[num_cols + cat_cols])
    metrics = {}
    for cond in ["ground","flight"]:
        sub = df_b[df_b["condition"]==cond]
        if not sub.empty:
            metrics[f"domain_model_base_{cond}"] = met(sub[TARGET].values, sub["pred_domain"].values)
            metrics[f"domain_model_base_{cond}"]["n"] = int(len(sub))
        else:
            metrics[f"domain_model_base_{cond}"] = {"note":"no rows"}
    return metrics

def eval_traj(df_traj_rows, colname):
    if not df_traj_rows:
        return {"note":"no comparable rows"}
    df_eval = pd.DataFrame(df_traj_rows)
    if colname not in df_eval.columns:
        return {"note":"no comparable rows"}
    mask = (~df_eval["actual_flight_day3"].isna()) & (~df_eval[colname].isna())
    y_true = df_eval.loc[mask, "actual_flight_day3"].values
    y_pred = df_eval.loc[mask, colname].values
    if len(y_true)==0:
        return {"note":"no comparable rows"}
    return met(y_true, y_pred)

def traj_linear(df, mean_reps=False):
    rows = []
    for mat in sorted(df["BaseMaterial"].dropna().unique()):
        g = df[(df["condition"]=="ground") & (df["BaseMaterial"]==mat)]
        f = df[(df["condition"]=="flight") & (df["BaseMaterial"]==mat)]
        if mean_reps:
            g = g.groupby("DayInt", as_index=False)[TARGET].mean()
            f = f.groupby("DayInt", as_index=False)[TARGET].mean()
        if g["DayInt"].nunique()<2:
            continue
        Xg = g[["DayInt"]].values
        yg = g[TARGET].values
        lin = LinearRegression().fit(Xg, yg)
        pred_A = float(lin.predict(np.array([[3]]))[0])
        growth_rate = np.nan
        if (g["DayInt"]==1).any() and (g["DayInt"]==3).any():
            cov_d1 = g.loc[g["DayInt"]==1, TARGET].mean()
            cov_d3 = g.loc[g["DayInt"]==3, TARGET].mean()
            growth_rate = (cov_d3 - cov_d1)/2.0
        pred_B = np.nan
        actual_f3 = np.nan
        if not f.empty:
            f_d1 = f[f["DayInt"]==1]
            f_d3 = f[f["DayInt"]==3]
            if not f_d3.empty:
                actual_f3 = float(f_d3[TARGET].mean())
            if not f_d1.empty and not np.isnan(growth_rate):
                pred_B = float(f_d1[TARGET].mean() + 2.0*growth_rate)
        rows.append({"BaseMaterial":mat,"pred_A":pred_A,"pred_B":pred_B,"actual_flight_day3":actual_f3})
    return rows

def traj_quadratic(df):
    from numpy.polynomial.polynomial import polyfit, polyval
    rows = []
    for mat in sorted(df["BaseMaterial"].dropna().unique()):
        g = df[(df["condition"]=="ground") & (df["BaseMaterial"]==mat)]
        f = df[(df["condition"]=="flight") & (df["BaseMaterial"]==mat)]
        if g["DayInt"].nunique()<2:
            continue
        xg = g["DayInt"].values.astype(float)
        yg = g[TARGET].values.astype(float)
        coefs = polyfit(xg, yg, deg=2)
        pred_A = float(polyval(3.0, coefs))
        growth_rate = np.nan
        if (g["DayInt"]==1).any() and (g["DayInt"]==3).any():
            cov_d1 = g.loc[g["DayInt"]==1, TARGET].mean()
            cov_d3 = g.loc[g["DayInt"]==3, TARGET].mean()
            growth_rate = (cov_d3 - cov_d1)/2.0
        pred_B = np.nan
        actual_f3 = np.nan
        if not f.empty:
            f_d1 = f[f["DayInt"]==1]
            f_d3 = f[f["DayInt"]==3]
            if not f_d3.empty:
                actual_f3 = float(f_d3[TARGET].mean())
            if not f_d1.empty and not np.isnan(growth_rate):
                pred_B = float(f_d1[TARGET].mean() + 2.0*growth_rate)
        rows.append({"BaseMaterial":mat,"pred_A":pred_A,"pred_B":pred_B,"actual_flight_day3":actual_f3})
    return rows

def traj_exponential(df):
    rows = []
    for mat in sorted(df["BaseMaterial"].dropna().unique()):
        g = df[(df["condition"]=="ground") & (df["BaseMaterial"]==mat)]
        f = df[(df["condition"]=="flight") & (df["BaseMaterial"]==mat)]
        if g["DayInt"].nunique()<2:
            continue
        xg = g["DayInt"].values.astype(float)
        yg = g[TARGET].values.astype(float)
        coefs = np.polyfit(xg, np.log1p(yg), 1)
        pred_A = float(np.expm1(np.polyval(coefs, 3.0)))
        growth_rate = np.nan
        if (g["DayInt"]==1).any() and (g["DayInt"]==3).any():
            cov_d1 = g.loc[g["DayInt"]==1, TARGET].mean()
            cov_d3 = g.loc[g["DayInt"]==3, TARGET].mean()
            growth_rate = (cov_d3 - cov_d1)/2.0
        pred_B = np.nan
        actual_f3 = np.nan
        if not f.empty:
            f_d1 = f[f["DayInt"]==1]
            f_d3 = f[f["DayInt"]==3]
            if not f_d3.empty:
                actual_f3 = float(f_d3[TARGET].mean())
            if not f_d1.empty and not np.isnan(growth_rate):
                pred_B = float(f_d1[TARGET].mean() + 2.0*growth_rate)
        rows.append({"BaseMaterial":mat,"pred_A":pred_A,"pred_B":pred_B,"actual_flight_day3":actual_f3})
    return rows

def baseline_models(df):
    def rf_orig():
        return RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    base_g2f = run_g2f(df, rf_orig, dict(include_dayint=False, add_interactions=False))
    base_dom = run_domain_aware(df, lambda: RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1))
    lin_rows = traj_linear(df, mean_reps=False)
    base_trajA = eval_traj(lin_rows, "pred_A")
    base_trajB = eval_traj(lin_rows, "pred_B")
    out = {}
    out.update(base_g2f)
    out.update(base_dom)
    out["trajectory_base_methodA"] = base_trajA
    out["trajectory_base_methodB"] = base_trajB
    return out

def exp_rf_tuned(df):
    def rf_tuned():
        return RandomForestRegressor(
            n_estimators=1000, max_depth=10, min_samples_leaf=2, max_features="sqrt",
            random_state=42, n_jobs=-1
        )
    g2f = run_g2f(df, rf_tuned, dict(include_dayint=False, add_interactions=False))
    dom = run_domain_aware(df, rf_tuned)
    return {"RF_tuned": {**g2f, **dom}}

def exp_gb(df):
    def gb():
        return GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=3, random_state=42)
    g2f = run_g2f(df, gb, dict(include_dayint=False, add_interactions=False))
    dom = run_domain_aware(df, gb)
    return {"GB": {**g2f, **dom}}

def exp_feat_interactions(df):
    def rf_orig():
        return RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    g2f = run_g2f(df.copy(), rf_orig, dict(include_dayint=False, add_interactions=True))
    dom = run_domain_aware(df.copy(), lambda: RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1))
    return {"feat_interactions": {**g2f, **dom}}

def exp_feat_add_dayint_g2f(df):
    def rf_orig():
        return RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    g2f = run_g2f(df.copy(), rf_orig, dict(include_dayint=True, add_interactions=False))
    dom = run_domain_aware(df.copy(), lambda: RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1))
    return {"feat_add_dayint_g2f": {**g2f, **dom}}

def exp_traj_quadratic(df):
    rows = traj_quadratic(df.copy())
    return {"traj_quadratic": {
        "trajectory_base_methodA": eval_traj(rows, "pred_A"),
        "trajectory_base_methodB": eval_traj(rows, "pred_B"),
    }}

def exp_traj_exponential(df):
    rows = traj_exponential(df.copy())
    return {"traj_exponential": {
        "trajectory_base_methodA": eval_traj(rows, "pred_A"),
        "trajectory_base_methodB": eval_traj(rows, "pred_B"),
    }}

def exp_traj_replicate_mean(df):
    rows = traj_linear(df.copy(), mean_reps=True)
    return {"traj_replicate_mean": {
        "trajectory_base_methodA": eval_traj(rows, "pred_A"),
        "trajectory_base_methodB": eval_traj(rows, "pred_B"),
    }}

def exp_xgboost(df):
    if not HAS_XGB:
        return {"XGBoost": {"g2f_regression": {"note":"xgboost not installed"},
                            "domain_model_base_ground": {"note":"xgboost not installed"},
                            "domain_model_base_flight": {"note":"xgboost not installed"}}}
    def xgb_ctor():
        return xgb.XGBRegressor(
            n_estimators=800, learning_rate=0.05, max_depth=5,
            subsample=0.9, colsample_bytree=0.8, random_state=42, n_jobs=-1,
            reg_lambda=1.0
        )
    g2f = run_g2f(df, xgb_ctor, dict(include_dayint=False, add_interactions=False))
    dom = run_domain_aware(df, xgb_ctor)
    return {"XGBoost": {**g2f, **dom}}

def exp_hist_gb(df):
    def hgb():
        return HistGradientBoostingRegressor(max_depth=8, learning_rate=0.05, max_iter=500, random_state=42)
    g2f = run_g2f(df, hgb, dict(include_dayint=False, add_interactions=False))
    dom = run_domain_aware(df, hgb)
    return {"HistGradientBoosting": {**g2f, **dom}}

def exp_cv_rf(df):
    # CV on G2F and Domain-aware
    def cv_rf_g2f():
        df_g = df[(df["condition"]=="ground") & df[TARGET].notna()].copy()
        df_f = df[(df["condition"]=="flight") & df[TARGET].notna()].copy()
        if df_g.empty or df_f.empty:
            return {"g2f_regression": {"note":"Insufficient ground/flight rows"}}

        df_aug, feats = get_dynamic_features_for_g2f(df.copy(), include_dayint=False, add_interactions=False)
        if not feats:
            return {"g2f_regression": {"note":"No usable features"}}

        Xg, yg = df_g[feats].values, df_g[TARGET].values
        Xf, yf = df_f[feats].values, df_f[TARGET].values

        base = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
        grid = GridSearchCV(
            estimator=base,
            param_grid={"max_depth":[5,10,None], "min_samples_leaf":[1,2,4], "max_features":[None,"sqrt"]},
            scoring="r2",
            cv=KFold(n_splits=3, shuffle=True, random_state=42)
        )
        grid.fit(Xg, yg)
        best = grid.best_estimator_
        ypred = best.predict(Xf)
        out = met(yf, ypred)
        out.update({"cv_best_params": grid.best_params_})
        return {"g2f_regression": out}

    def cv_rf_domain():
        df_b = df[df[TARGET].notna()].copy()
        if df_b.empty:
            return {"domain_model_base_ground":{"note":"no rows"},"domain_model_base_flight":{"note":"no rows"}}
        df_b["condition_bin"] = (df_b["condition"]=="flight").astype(int)
        num_candidates = df_b.select_dtypes(include=[np.number]).columns.tolist()
        num_candidates = [c for c in num_candidates if c != TARGET]
        num_cols = [c for c in num_candidates if c not in ["DayInt","condition_bin"]]
        for c in ["DayInt","condition_bin"]:
            if c in df_b.columns and c not in num_cols:
                num_cols.append(c)
        cat_cols = ["BaseMaterial"] if "BaseMaterial" in df_b.columns else []

        pre = ColumnTransformer(
            transformers=[
                ("num", StandardScaler(), num_cols),
                ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
            ],
            remainder="drop"
        )
        base = RandomForestRegressor(n_estimators=500, random_state=42, n_jobs=-1)
        pipe = Pipeline([("pre", pre), ("model", base)])
        grid = GridSearchCV(
            estimator=pipe,
            param_grid={
                "model__max_depth":[5,10,None],
                "model__min_samples_leaf":[1,2,4],
                "model__max_features":[None,"sqrt"]
            },
            scoring="r2",
            cv=KFold(n_splits=3, shuffle=True, random_state=42)
        )
        grid.fit(df_b[num_cols + cat_cols], df_b[TARGET])
        df_b["pred_cv"] = grid.predict(df_b[num_cols + cat_cols])
        metrics = {}
        for cond in ["ground","flight"]:
            sub = df_b[df_b["condition"]==cond]
            if not sub.empty:
                m = met(sub[TARGET].values, sub["pred_cv"].values)
                m["cv_best_params"] = grid.best_params_
                m["n"] = int(len(sub))
                metrics[f"domain_model_base_{cond}"] = m
            else:
                metrics[f"domain_model_base_{cond}"] = {"note":"no rows"}
        return metrics

    out = {}
    out.update(cv_rf_g2f())
    out.update(cv_rf_domain())
    return {"CV_RandomForest": out}

def exp_combined_global(df):

    metrics = {}
    if not HAS_STATSMODELS:
        return {"Combined_Global": {
            "domain_model_base_ground":{"note":"statsmodels not installed"},
            "domain_model_base_flight":{"note":"statsmodels not installed"},
            "trajectory_combined_methodA":{"note":"statsmodels not installed"}
        }}
    df_b = df[df[TARGET].notna()].copy()
    if df_b.empty:
        return {"Combined_Global": {
            "domain_model_base_ground":{"note":"no rows"},
            "domain_model_base_flight":{"note":"no rows"},
            "trajectory_combined_methodA":{"note":"no rows"}
        }}

    df_b["condition_bin"] = (df_b["condition"]=="flight").astype(int)

    # Patsy formula with Q() escaping for the target
    formula = f'Q("{TARGET}") ~ DayInt * condition_bin + C(BaseMaterial)'

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = smf.ols(formula, data=df_b).fit()

    df_b["pred_ols"] = model.predict(df_b)

    for cond in ["ground","flight"]:
        sub = df_b[df_b["condition"]==cond]
        if not sub.empty:
            y_true = sub[TARGET].values
            y_pred = sub["pred_ols"].values
            metrics[f"domain_model_base_{cond}"] = met(y_true, y_pred)
            metrics[f"domain_model_base_{cond}"]["n"] = int(len(sub))
        else:
            metrics[f"domain_model_base_{cond}"] = {"note":"no rows"}

    # Predict Flight Day3 per BaseMaterial and evaluate vs actual
    rows = []
    for mat in sorted(df_b["BaseMaterial"].dropna().unique()):
        df_pred = pd.DataFrame({"DayInt":[3], "condition_bin":[1], "BaseMaterial":[mat]})
        yhat = float(model.predict(df_pred)[0])
        f_d3 = df_b[(df_b["BaseMaterial"]==mat) & (df_b["condition"]=="flight") & (df_b["DayInt"]==3)]
        actual = float(f_d3[TARGET].mean()) if not f_d3.empty else np.nan
        rows.append({"BaseMaterial": mat, "pred_ols_day3_flight": yhat, "actual_flight_day3": actual})

    df_eval = pd.DataFrame(rows)
    mask = (~df_eval["actual_flight_day3"].isna()) & (~df_eval["pred_ols_day3_flight"].isna())
    if mask.any():
        metrics["trajectory_combined_methodA"] = met(
            df_eval.loc[mask,"actual_flight_day3"].values,
            df_eval.loc[mask,"pred_ols_day3_flight"].values
        )
        metrics["trajectory_combined_methodA"]["n"] = int(mask.sum())
    else:
        metrics["trajectory_combined_methodA"] = {"note":"no comparable rows"}

    return {"Combined_Global": metrics}

In [ ]:
def safe_run(name, fn, *args, **kwargs):
    """Run an experiment function safely; on error, return a note."""
    try:
        out = fn(*args, **kwargs)
        if not isinstance(out, dict) or len(out) == 0:
            return {name: {"_runner_error": {"note": "empty or invalid experiment output"}}}
        return out
    except Exception as e:
        return {name: {"_runner_error": {"note": f"exception: {type(e).__name__}: {e}"}}}

df = load_clean_data()
baseline = baseline_models(df)

saved_baseline = {}
metrics_json_path = ROOT_OUT / "metrics.json"
if metrics_json_path.exists():
    try:
        saved_baseline = json.load(open(metrics_json_path, "r"))
    except Exception:
        saved_baseline = {}

# Collect experiments with robust wrapper
experiments = {}
# prior
experiments.update(safe_run("RF_tuned", exp_rf_tuned, df))
experiments.update(safe_run("GB", exp_gb, df))
experiments.update(safe_run("feat_interactions", exp_feat_interactions, df))
experiments.update(safe_run("feat_add_dayint_g2f", exp_feat_add_dayint_g2f, df))
experiments.update(safe_run("traj_quadratic", exp_traj_quadratic, df))
experiments.update(safe_run("traj_exponential", exp_traj_exponential, df))
experiments.update(safe_run("traj_replicate_mean", exp_traj_replicate_mean, df))
# new
experiments.update(safe_run("XGBoost", exp_xgboost, df))
experiments.update(safe_run("HistGradientBoosting", exp_hist_gb, df))
experiments.update(safe_run("CV_RandomForest", exp_cv_rf, df))
experiments.update(safe_run("Combined_Global", exp_combined_global, df))

def metrics_to_rows(experiment_label, metrics_block):
    """
    experiment_label: the experiment name to put in the 'experiment' column
    metrics_block: dict like {"g2f_regression": {...}, "domain_model_base_ground": {...}, ...}
    """
    rows = []
    for metric_name, val in metrics_block.items():
        if isinstance(val, dict) and ("rmse" in val or "note" in val):
            row = val.copy()
            row["experiment"] = experiment_label
            row["metric"] = metric_name
            rows.append(row)
    return rows

rows = []

# Baselines
rows += metrics_to_rows("baseline_recomputed", baseline)
if saved_baseline:
    rows += metrics_to_rows("baseline_saved", saved_baseline)

# Experiments (use OUTER KEY as the experiment label)
captured_experiments = []
for exp_key, metrics_block in experiments.items():
    captured_experiments.append(exp_key)
    rows += metrics_to_rows(exp_key, metrics_block)

cmp_df = pd.DataFrame(rows)

# Sort for readability
order = [
    "g2f_regression",
    "domain_model_base_ground", "domain_model_base_flight",
    "trajectory_base_methodA", "trajectory_base_methodB",
    "trajectory_combined_methodA"
]
if "metric" in cmp_df.columns:
    cmp_df["metric"] = pd.Categorical(cmp_df["metric"], categories=order, ordered=True)
    cmp_df = cmp_df.sort_values(["metric","experiment"]).reset_index(drop=True)

# Save comparison outputs
cmp_json = EXP_OUT / "metrics_comparison.json"
cmp_csv = EXP_OUT / "metrics_comparison.csv"
cmp_df.to_json(cmp_json, orient="records", indent=2)
cmp_df.to_csv(cmp_csv, index=False)

# Log which experiments were captured
with open(EXP_OUT / "experiments_log.txt", "w") as fh:
    fh.write("Captured experiments (experiment column labels):\n")
    for lab in captured_experiments:
        fh.write(f"- {lab}\n")

print("Done. Wrote:")
print("-", cmp_json)
print("-", cmp_csv)
print("-", EXP_OUT / "experiments_log.txt")

def best_by(metric_name, stat_key, better="max"):
    sub = cmp_df[(cmp_df["metric"]==metric_name) & (~cmp_df[stat_key].isna())]
    if sub.empty:
        return {"experiment": None, stat_key: None}
    idx = sub[stat_key].idxmax() if better=="max" else sub[stat_key].idxmin()
    return {"experiment": sub.loc[idx, "experiment"], stat_key: float(sub.loc[idx, stat_key])}

print("\n=== winners per metric ===")
for m in order:
    if m not in cmp_df["metric"].unique():
        print(f"{m}: (no rows)")
        continue
    w_r2 = best_by(m, "r2", better="max")
    w_rmse = best_by(m, "rmse", better="min")
    print(f"{m}: best R2 → {w_r2}, lowest RMSE → {w_rmse}")


Done. Wrote:
- /content/lsds55_outputs/experiments/metrics_comparison.json
- /content/lsds55_outputs/experiments/metrics_comparison.csv
- /content/lsds55_outputs/experiments/experiments_log.txt

=== Quick winners per metric ===
g2f_regression: best R2 → {'experiment': 'CV_RandomForest', 'r2': 0.9687976252136771}, lowest RMSE → {'experiment': 'CV_RandomForest', 'rmse': 6.951043703668633}
domain_model_base_ground: best R2 → {'experiment': 'XGBoost', 'r2': 0.9999999992018583}, lowest RMSE → {'experiment': 'XGBoost', 'rmse': 0.000921669140147205}
domain_model_base_flight: best R2 → {'experiment': 'XGBoost', 'r2': 0.9999999996688176}, lowest RMSE → {'experiment': 'XGBoost', 'rmse': 0.000716126590920895}
trajectory_base_methodA: best R2 → {'experiment': 'baseline_recomputed', 'r2': -0.2585982876309145}, lowest RMSE → {'experiment': 'baseline_recomputed', 'rmse': 43.47417728653114}
trajectory_base_methodB: best R2 → {'experiment': 'baseline_recomputed', 'r2': 0.03753894267906133}, lowest RMSE

In [ ]:
# -*- coding: utf-8 -*-
"""
Hybrid Mixture-of-Experts (MoE) for numeric only data
"""

import re, json, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression

EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
OUTDIR = Path("/content/lsds55_outputs/moe")
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):
    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue
        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]
        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true)==0 or len(y_pred)==0:
        return {"note":"no rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae":  float(mean_absolute_error(y_true, y_pred)),
        "r2":   float(r2_score(y_true, y_pred)) if len(np.unique(y_true))>1 else np.nan,
        "n":    int(len(y_true))
    }

xl = pd.ExcelFile(EXCEL_PATH)
dfs = []
if "GROUND DATA" in xl.sheet_names:
    g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
    g = normalize_headers(g).dropna(how="all")
    g["condition"] = "ground"
    dfs.append(g)
if "FLIGHT DATA" in xl.sheet_names:
    f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
    f = normalize_headers(f).dropna(how="all")
    f["condition"] = "flight"
    dfs.append(f)
if not dfs:
    raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' found.")

df_all = pd.concat(dfs, ignore_index=True)
df_all = clean_cols(df_all)

cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)

if "Material and Incubation day" not in df_all.columns:
    raise RuntimeError("Missing 'Material and Incubation day' after normalization.")

df_all[["Material","day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
df_all[["BaseMaterial","DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))
df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])

df = df_all.dropna(subset=["BaseMaterial","DayInt","condition", TARGET]).copy()
df["condition_bin"] = (df["condition"]=="flight").astype(int)

# Numeric features for Expert A (domain-aware)
num_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
# Drop target & pure identifier-like columns
numA = [c for c in num_candidates if c not in {TARGET}]
# Keep DayInt & condition_bin; others are whatever numeric exist (mass/thickness/roughness)
catA = ["BaseMaterial"]

# Expert B features (trajectory-aware linear): DayInt, DayInt*condition, BaseMaterial
df["DayInt_x_condition"] = df["DayInt"] * df["condition_bin"]
numB = ["DayInt", "DayInt_x_condition"]
catB = ["BaseMaterial"]

# For the gate we’ll use a slim, robust set
gate_num = []
for c in ["DayInt", "condition_bin"]:
    if c in df.columns:
        gate_num.append(c)
# add a couple of stable numeric morphology features if present
for c in ["Biofilm mass (µm^3/µm^2)","roughness coefficient Ra*","Biofilm Maximum thickness (µm)","Biofilm biomass mean thickness (µm)"]:
    if c in df.columns:
        gate_num.append(c)
gate_cat = ["BaseMaterial"]

preA = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numA),  # with_mean=False keeps sparse-safe
        ("cat", OneHotEncoder(handle_unknown="ignore"), catA)
    ],
    remainder="drop"
)
rf_base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
rf_grid = {
    "model__max_depth": [5, 10, None],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": [None, "sqrt"]
}
pipeA = Pipeline([("pre", preA), ("model", rf_base)])
cvA = GridSearchCV(
    pipeA, rf_grid, cv=KFold(n_splits=3, shuffle=True, random_state=42),
    scoring="r2", n_jobs=-1
)

preB = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numB),
        ("cat", OneHotEncoder(handle_unknown="ignore"), catB)
    ],
    remainder="drop"
)
pipeB = Pipeline([("pre", preB), ("model", LinearRegression())])

# Build OOF predictions for both experts for gating labels
kfold = KFold(n_splits=3, shuffle=True, random_state=42)
oof_idx = []
oof_pred_A = np.zeros(len(df))
oof_pred_B = np.zeros(len(df))

X_A = df[numA + catA].copy()
X_B = df[numB + catB].copy()
y = df[TARGET].values

for train_idx, val_idx in kfold.split(df):
    # Expert A
    cvA.fit(X_A.iloc[train_idx], y[train_idx])
    predA = cvA.predict(X_A.iloc[val_idx])
    oof_pred_A[val_idx] = predA
    # Expert B
    pipeB.fit(X_B.iloc[train_idx], y[train_idx])
    predB = pipeB.predict(X_B.iloc[val_idx])
    oof_pred_B[val_idx] = predB
    oof_idx.extend(val_idx.tolist())

# OOF rows may be out of order; ensure alignment
oof_mask = np.zeros(len(df), dtype=bool)
oof_mask[oof_idx] = True

# Gating labels: 1 if Expert B is better (lower abs error), else 0
abs_err_A = np.abs(y - oof_pred_A)
abs_err_B = np.abs(y - oof_pred_B)
gate_labels = (abs_err_B < abs_err_A).astype(int)

cvA.fit(X_A, y)  # Expert A finalized
pipeB.fit(X_B, y)  # Expert B finalized

preGate = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), gate_num),
        ("cat", OneHotEncoder(handle_unknown="ignore"), gate_cat)
    ],
    remainder="drop"
)
gate = Pipeline([
    ("pre", preGate),
    ("clf", LogisticRegression(max_iter=400, solver="lbfgs"))
])
X_gate = df[gate_num + gate_cat].copy()
gate.fit(X_gate, gate_labels)

predA_all = cvA.predict(X_A)
predB_all = pipeB.predict(X_B)
wB = gate.predict_proba(X_gate)[:,1]  # weight for Expert B
pred_mix = (1.0 - wB) * predA_all + wB * predB_all

def add_metrics(prefix, y_true, y_hat, ddict):
    ddict[prefix] = met(y_true, y_hat)

metrics = {}
add_metrics("ExpertA_CVRandomForest_all", y, predA_all, metrics)
add_metrics("ExpertB_LinearTrajectory_all", y, predB_all, metrics)
add_metrics("Mixture_all", y, pred_mix, metrics)

# Per condition
for cond in ["ground","flight"]:
    idx = (df["condition"]==cond).values
    add_metrics(f"ExpertA_CVRandomForest_{cond}", y[idx], predA_all[idx], metrics)
    add_metrics(f"ExpertB_LinearTrajectory_{cond}", y[idx], predB_all[idx], metrics)
    add_metrics(f"Mixture_{cond}", y[idx], pred_mix[idx], metrics)

pred_out = df[["Material and Incubation day","BaseMaterial","DayInt","condition",TARGET]].copy()
pred_out["pred_expertA"] = predA_all
pred_out["pred_expertB"] = predB_all
pred_out["gate_wB"] = wB
pred_out["pred_mixture"] = pred_mix
pred_out.to_csv(OUTDIR / "moe_predictions.csv", index=False)

json.dump(metrics, open(OUTDIR / "moe_metrics.json","w"), indent=2)

# Gate report (top coefficients if linear-ish)
gate_report = {"note": "gate is LogisticRegression over preprocessed features"}
try:
    # Recover feature names from ColumnTransformer
    ohe = gate.named_steps["pre"].transformers_[1][1]  # ("cat", OneHotEncoder, gate_cat)
    num_names = gate_num
    cat_names = []
    if hasattr(ohe, "get_feature_names_out"):
        cat_names = list(ohe.get_feature_names_out(gate_cat))
    feat_names = list(num_names) + list(cat_names)
    coef = gate.named_steps["clf"].coef_.ravel()
    top_idx = np.argsort(np.abs(coef))[::-1][:15]
    gate_report = {
        "top_features": [
            {"feature": feat_names[i] if i < len(feat_names) else f"feat_{i}",
             "coef": float(coef[i])}
            for i in top_idx
        ],
        "intercept": float(gate.named_steps["clf"].intercept_[0])
    }
except Exception as e:
    gate_report["warn"] = f"feature inspection failed: {e}"

json.dump(gate_report, open(OUTDIR / "gate_report.json","w"), indent=2)

print("MoE done. Wrote:")
print("-", OUTDIR / "moe_predictions.csv")
print("-", OUTDIR / "moe_metrics.json")
print("-", OUTDIR / "gate_report.json")

# (optional) quick console summary
def pretty(m):
    return {k: {kk: round(vv,4) if isinstance(vv, float) else vv for kk,vv in d.items()} for k,d in m.items()}
print(json.dumps(pretty(metrics), indent=2))


MoE done. Wrote:
- /content/lsds55_outputs/moe/moe_predictions.csv
- /content/lsds55_outputs/moe/moe_metrics.json
- /content/lsds55_outputs/moe/gate_report.json
{
  "ExpertA_CVRandomForest_all": {
    "rmse": 1.8698,
    "mae": 1.1574,
    "r2": 0.9975,
    "n": 36
  },
  "ExpertB_LinearTrajectory_all": {
    "rmse": 17.0,
    "mae": 13.9968,
    "r2": 0.7953,
    "n": 36
  },
  "Mixture_all": {
    "rmse": 3.4911,
    "mae": 2.8556,
    "r2": 0.9914,
    "n": 36
  },
  "ExpertA_CVRandomForest_ground": {
    "rmse": 2.1731,
    "mae": 1.423,
    "r2": 0.9956,
    "n": 18
  },
  "ExpertB_LinearTrajectory_ground": {
    "rmse": 18.7451,
    "mae": 15.5832,
    "r2": 0.6699,
    "n": 18
  },
  "Mixture_ground": {
    "rmse": 3.0375,
    "mae": 2.4501,
    "r2": 0.9913,
    "n": 18
  },
  "ExpertA_CVRandomForest_flight": {
    "rmse": 1.5067,
    "mae": 0.8919,
    "r2": 0.9985,
    "n": 18
  },
  "ExpertB_LinearTrajectory_flight": {
    "rmse": 15.0541,
    "mae": 12.4105,
    "r2": 0.853

In [ ]:
# Trajectory plots random forest for expert A
import os
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path("/content/lsds55_outputs/moe")
PRED_CSV = ROOT / "moe_predictions.csv"
OUTDIR = ROOT / "trajectories_expertA"
OUTDIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(PRED_CSV)
# Basic hygiene
df["DayInt"] = pd.to_numeric(df["DayInt"], errors="coerce")
df = df.dropna(subset=["BaseMaterial","DayInt","condition","Biofilm surface area coverage (%)","pred_expertA"])
df["condition"] = df["condition"].str.lower().str.strip()

# Helper: safe filename from material name
def safe_name(s):
    return re.sub(r"[^A-Za-z0-9]+", "_", str(s)).strip("_")

materials = sorted(df["BaseMaterial"].unique())

for mat in materials:
    sub = df[df["BaseMaterial"] == mat].copy()

    plt.figure(figsize=(6,4.2))
    # Actual points (scatter) and their per-day means (dashed line)
    for cond, marker in [("ground","o"), ("flight","s")]:
        csub = sub[sub["condition"]==cond]
        if csub.empty:
            continue
        # scatter all replicates
        plt.scatter(csub["DayInt"], csub["Biofilm surface area coverage (%)"],
                    label=f"{cond.capitalize()} actual", marker=marker, alpha=0.8)
        # day-wise mean of actuals
        m_actual = csub.groupby("DayInt")["Biofilm surface area coverage (%)"].mean().sort_index()
        if len(m_actual) > 1:
            plt.plot(m_actual.index.values, m_actual.values, linestyle="--", alpha=0.7)

    # Expert A predictions: per-day mean line per condition
    for cond in ["ground","flight"]:
        csub = sub[sub["condition"]==cond]
        if csub.empty:
            continue
        m_pred = csub.groupby("DayInt")["pred_expertA"].mean().sort_index()
        plt.plot(m_pred.index.values, m_pred.values, linewidth=2, label=f"{cond.capitalize()} ExpertA (mean)")

    plt.title(f"Expert A Trajectory — {mat}")
    plt.xlabel("Day")
    plt.ylabel("Biofilm surface area coverage (%)")
    plt.xticks(sorted(sub["DayInt"].unique()))
    plt.legend()
    plt.tight_layout()
    plt.savefig(OUTDIR / f"expertA_trajectory_{safe_name(mat)}.png", dpi=150)
    plt.close()

print(f"Saved per-material plots to: {OUTDIR}")

n = len(materials)
cols = 3
rows = int(np.ceil(n / cols))
fig, axes = plt.subplots(rows, cols, figsize=(cols*5.0, rows*3.6), squeeze=False)

for idx, mat in enumerate(materials):
    r, c = divmod(idx, cols)
    ax = axes[r, c]
    sub = df[df["BaseMaterial"]==mat]
    # Actual points
    for cond, marker in [("ground","o"), ("flight","s")]:
        csub = sub[sub["condition"]==cond]
        if csub.empty:
            continue
        ax.scatter(csub["DayInt"], csub["Biofilm surface area coverage (%)"], marker=marker, alpha=0.8, label=f"{cond[:1].upper()} act")
        m_actual = csub.groupby("DayInt")["Biofilm surface area coverage (%)"].mean().sort_index()
        if len(m_actual)>1:
            ax.plot(m_actual.index.values, m_actual.values, linestyle="--", alpha=0.6)
    # Expert A mean preds
    for cond in ["ground","flight"]:
        csub = sub[sub["condition"]==cond]
        if csub.empty:
            continue
        m_pred = csub.groupby("DayInt")["pred_expertA"].mean().sort_index()
        ax.plot(m_pred.index.values, m_pred.values, linewidth=2, label=f"{cond[:1].upper()} A")

    ax.set_title(mat)
    ax.set_xlabel("Day")
    ax.set_ylabel("Coverage (%)")
    ax.set_xticks(sorted(sub["DayInt"].unique()))
    ax.grid(False)

# tidy legends: only on last axis
for r in range(rows):
    for c in range(cols):
        if r*cols + c >= n:
            axes[r, c].axis("off")
# Put a single legend
handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=6, frameon=False)
fig.tight_layout(rect=[0,0,1,0.95])
combined_path = OUTDIR / "expertA_trajectories_all_materials.png"
fig.savefig(combined_path, dpi=150)
plt.close(fig)

print(f"Also saved combined grid: {combined_path}")


Saved per-material plots to: /content/lsds55_outputs/moe/trajectories_expertA
Also saved combined grid: /content/lsds55_outputs/moe/trajectories_expertA/expertA_trajectories_all_materials.png


In [ ]:
# -*- coding: utf-8 -*-
"""
blind testing moe model 

"""

import re, json, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression

EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
OUTDIR = Path("/content/lsds55_outputs/moe_blind")
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):
    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue
        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]
        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true)==0 or len(y_pred)==0:
        return {"note":"no rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae":  float(mean_absolute_error(y_true, y_pred)),
        "r2":   float(r2_score(y_true, y_pred)) if len(np.unique(y_true))>1 else np.nan,
        "n":    int(len(y_true))
    }

def load_clean_df():
    xl = pd.ExcelFile(EXCEL_PATH)
    dfs = []
    if "GROUND DATA" in xl.sheet_names:
        g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
        g = normalize_headers(g).dropna(how="all")
        g["condition"] = "ground"
        dfs.append(g)
    if "FLIGHT DATA" in xl.sheet_names:
        f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
        f = normalize_headers(f).dropna(how="all")
        f["condition"] = "flight"
        dfs.append(f)
    if not dfs:
        raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' found.")
    df_all = pd.concat(dfs, ignore_index=True)
    df_all = clean_cols(df_all)
    cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
    df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)
    if "Material and Incubation day" not in df_all.columns:
        raise RuntimeError("Missing 'Material and Incubation day' after normalization.")
    df_all[["Material","day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
    df_all[["BaseMaterial","DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))
    df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])
    df = df_all.dropna(subset=["BaseMaterial","DayInt","condition", TARGET]).copy()
    df["condition_bin"] = (df["condition"]=="flight").astype(int)
    df["DayInt"] = pd.to_numeric(df["DayInt"], errors="coerce")
    df["DayInt_x_condition"] = df["DayInt"] * df["condition_bin"]
    return df

def fit_moe(train_df):
    """Fit MoE on train_df, return dict with trained pipelines and feature lists."""
    df = train_df.copy()

    # Expert A feature sets
    num_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
    numA = [c for c in num_candidates if c != TARGET]
    catA = ["BaseMaterial"]

    # Expert B (trajectory)
    numB = ["DayInt", "DayInt_x_condition"]
    numB = [c for c in numB if c in df.columns]
    catB = ["BaseMaterial"]

    # Gate features: small, robust set
    gate_num = []
    for c in ["DayInt", "condition_bin"]:
        if c in df.columns:
            gate_num.append(c)
    for c in ["Biofilm mass (µm^3/µm^2)", "roughness coefficient Ra*", "Biofilm Maximum thickness (µm)", "Biofilm biomass mean thickness (µm)"]:
        if c in df.columns:
            gate_num.append(c)
    gate_cat = ["BaseMaterial"]

    y = df[TARGET].values

    # Expert A: CV-tuned RF
    preA = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), numA),
            ("cat", OneHotEncoder(handle_unknown="ignore"), catA)
        ],
        remainder="drop"
    )
    rf_base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
    rf_grid = {
        "model__max_depth": [5, 10, None],
        "model__min_samples_leaf": [1, 2, 4],
        "model__max_features": [None, "sqrt"]
    }
    pipeA = Pipeline([("pre", preA), ("model", rf_base)])
    cvA = GridSearchCV(
        pipeA, rf_grid,
        cv=KFold(n_splits=3, shuffle=True, random_state=42),
        scoring="r2", n_jobs=-1
    )

    # Expert B: linear trajectory
    preB = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), numB),
            ("cat", OneHotEncoder(handle_unknown="ignore"), catB)
        ],
        remainder="drop"
    )
    pipeB = Pipeline([("pre", preB), ("model", LinearRegression())])

    # OOF preds for gate labels
    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    oof_pred_A = np.zeros(len(df))
    oof_pred_B = np.zeros(len(df))
    X_A = df[numA + catA].copy()
    X_B = df[numB + catB].copy()

    for train_idx, val_idx in kfold.split(df):
        cvA.fit(X_A.iloc[train_idx], y[train_idx])
        oof_pred_A[val_idx] = cvA.predict(X_A.iloc[val_idx])
        pipeB.fit(X_B.iloc[train_idx], y[train_idx])
        oof_pred_B[val_idx] = pipeB.predict(X_B.iloc[val_idx])

    abs_err_A = np.abs(y - oof_pred_A)
    abs_err_B = np.abs(y - oof_pred_B)
    gate_labels = (abs_err_B < abs_err_A).astype(int)

    # Fit experts on all training data
    cvA.fit(X_A, y)
    pipeB.fit(X_B, y)

    # Fit gate on training data
    preGate = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), gate_num),
            ("cat", OneHotEncoder(handle_unknown="ignore"), gate_cat)
        ],
        remainder="drop"
    )
    gate = Pipeline([
        ("pre", preGate),
        ("clf", LogisticRegression(max_iter=400, solver="lbfgs"))
    ])
    X_gate = df[gate_num + gate_cat].copy()
    gate.fit(X_gate, gate_labels)

    return {
        "cvA": cvA,
        "pipeB": pipeB,
        "gate": gate,
        "numA": numA,
        "catA": catA,
        "numB": numB,
        "catB": catB,
        "gate_num": gate_num,
        "gate_cat": gate_cat,
    }

def eval_moe(model, df, split_name):
    """Return metrics dict and prediction DataFrame for a given split."""
    d = df.copy()
    y = d[TARGET].values

    X_A = d[model["numA"] + model["catA"]].copy()
    X_B = d[model["numB"] + model["catB"]].copy()
    X_gate = d[model["gate_num"] + model["gate_cat"]].copy()

    predA = model["cvA"].predict(X_A)
    predB = model["pipeB"].predict(X_B)
    wB = model["gate"].predict_proba(X_gate)[:,1]
    predMix = (1.0 - wB) * predA + wB * predB

    metrics = {
        f"{split_name}_ExpertA": met(y, predA),
        f"{split_name}_ExpertB": met(y, predB),
        f"{split_name}_Mixture": met(y, predMix),
    }

    pred_df = d[["Material and Incubation day","BaseMaterial","DayInt","condition",TARGET]].copy()
    pred_df["pred_expertA"] = predA
    pred_df["pred_expertB"] = predB
    pred_df["gate_wB"] = wB
    pred_df["pred_mixture"] = predMix
    return metrics, pred_df

df_full = load_clean_df()

all_metrics = {}

train_df, test_df = train_test_split(
    df_full,
    test_size=0.3,
    random_state=42,
    shuffle=True,
)

moe_random = fit_moe(train_df)
m_train, pred_train = eval_moe(moe_random, train_df, "randomTrain")
m_test, pred_test = eval_moe(moe_random, test_df, "randomTest")

all_metrics.update(m_train)
all_metrics.update(m_test)

pred_test.to_csv(OUTDIR / "moe_blind_predictions_random_split.csv", index=False)

ground_df = df_full[df_full["condition"]=="ground"].copy()
flight_df = df_full[df_full["condition"]=="flight"].copy()

moe_g2f = fit_moe(ground_df)
m_ground_in, pred_ground_in = eval_moe(moe_g2f, ground_df, "G2F_train_ground")
m_flight_blind, pred_flight_blind = eval_moe(moe_g2f, flight_df, "G2F_test_flight")

all_metrics.update(m_ground_in)
all_metrics.update(m_flight_blind)

pred_flight_blind.to_csv(OUTDIR / "moe_blind_predictions_ground_to_flight.csv", index=False)

json.dump(all_metrics, open(OUTDIR / "moe_blind_metrics.json","w"), indent=2)

def pretty(m):
    out = {}
    for k,v in m.items():
        out[k] = {kk: (round(vv,4) if isinstance(vv,float) else vv) for kk,vv in v.items()}
    return out

print("Blind-test metrics (rounded):")
print(json.dumps(pretty(all_metrics), indent=2))
print("\nWrote:")
print("-", OUTDIR / "moe_blind_metrics.json")
print("-", OUTDIR / "moe_blind_predictions_random_split.csv")
print("-", OUTDIR / "moe_blind_predictions_ground_to_flight.csv")


Blind-test metrics (rounded):
{
  "randomTrain_ExpertA": {
    "rmse": 1.801,
    "mae": 1.247,
    "r2": 0.9978,
    "n": 25
  },
  "randomTrain_ExpertB": {
    "rmse": 13.7902,
    "mae": 11.4291,
    "r2": 0.8689,
    "n": 25
  },
  "randomTrain_Mixture": {
    "rmse": 4.492,
    "mae": 3.3786,
    "r2": 0.9861,
    "n": 25
  },
  "randomTest_ExpertA": {
    "rmse": 5.3952,
    "mae": 3.4357,
    "r2": 0.978,
    "n": 11
  },
  "randomTest_ExpertB": {
    "rmse": 23.7824,
    "mae": 20.4639,
    "r2": 0.5722,
    "n": 11
  },
  "randomTest_Mixture": {
    "rmse": 9.0319,
    "mae": 7.7858,
    "r2": 0.9383,
    "n": 11
  },
  "G2F_train_ground_ExpertA": {
    "rmse": 3.5242,
    "mae": 2.3066,
    "r2": 0.9883,
    "n": 18
  },
  "G2F_train_ground_ExpertB": {
    "rmse": 16.2495,
    "mae": 12.7293,
    "r2": 0.7519,
    "n": 18
  },
  "G2F_train_ground_Mixture": {
    "rmse": 4.6252,
    "mae": 3.6367,
    "r2": 0.9799,
    "n": 18
  },
  "G2F_test_flight_ExpertA": {
    "rmse": 8.

In [ ]:
# -*- coding: utf-8 -*-
"""
expert A only pipeline

"""

import re
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# ------------------------- Config -------------------------
EXCEL_PATH = Path("/content/LSDS-55_microscopy_LSDS-55_ConfocalMicroscopy_flores_SUBMITTED.xlsx")
OUTDIR = Path("/content/lsds55_outputs_expertA")
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"
EXPECTED_FEATURES = [
    "Biofilm mass (µm^3/µm^2)",
    "Biofilm biomass mean thickness (µm)",
    "Biofilm Maximum thickness (µm)",
    "roughness coefficient Ra*",
]
EXPECTED_COLS = [
    "Material and Incubation day",
    "sample ID",
    *EXPECTED_FEATURES,
    TARGET,
]

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:

    df = clean_cols(df)
    rename_map = {}
    for col in df.columns:
        c = col.lower()
        if "material" in c and "incubation" in c:
            rename_map[col] = "Material and Incubation day"
        elif c.startswith("sample"):
            rename_map[col] = "sample ID"
        elif "mass" in c and ("um" in c or "µm" in c):
            rename_map[col] = "Biofilm mass (µm^3/µm^2)"
        elif "mean thickness" in c:
            rename_map[col] = "Biofilm biomass mean thickness (µm)"
        elif "maximum thickness" in c:
            rename_map[col] = "Biofilm Maximum thickness (µm)"
        elif "surface area coverage" in c:
            rename_map[col] = "Biofilm surface area coverage (%)"
        elif "roughness" in c:
            rename_map[col] = "roughness coefficient Ra*"
    df = df.rename(columns=rename_map)
    keep = [c for c in EXPECTED_COLS if c in df.columns]
    return df[keep] if keep else df

def parse_material_day(s: str):
    if pd.isna(s):
        return (np.nan, np.nan)
    s_norm = re.sub(r"\s+", " ", str(s)).strip()
    m_day = re.search(r"day\s*([123])", s_norm, flags=re.IGNORECASE)
    day = int(m_day.group(1)) if m_day else np.nan
    mat = re.split(r"\bday\b", s_norm, flags=re.IGNORECASE)[0].strip()
    mat = mat.replace("Celullose", "Cellulose").strip()
    return (mat, day)

def parse_base_and_day_v2(x):

    if pd.isna(x):
        return (np.nan, np.nan)
    s = re.sub(r"\s+", " ", str(x)).strip()
    m = re.search(r"day\s*([123])\s*$", s, flags=re.IGNORECASE)
    day = int(m.group(1)) if m else np.nan
    base = re.sub(r"day\s*[123]\s*$", "", s, flags=re.IGNORECASE).strip()
    base = base.replace("Celullose", "Cellulose").strip()
    return (base, day)

def consolidate_duplicate_columns(df: pd.DataFrame, cols_to_merge: list) -> pd.DataFrame:

    df = df.copy()
    merged_info = []
    for name in cols_to_merge:
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) <= 1:
            continue
        dup_block = df.iloc[:, idxs]
        dup_block_num = dup_block.apply(pd.to_numeric, errors="coerce")
        merged_series = dup_block_num.bfill(axis=1).iloc[:, 0]
        if merged_series.isna().all():
            merged_series = dup_block.bfill(axis=1).iloc[:, 0]
        first_idx = idxs[0]
        df.iloc[:, first_idx] = merged_series
        df.drop(columns=df.columns[idxs[1:]], inplace=True)
        merged_info.append((name, len(idxs)))
    if merged_info:
        print("Consolidated duplicate columns:", merged_info)
    return df

def safe_coerce_numeric(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    """
    Coerce listed columns to numeric, dropping extra duplicates if any remain.
    """
    df = df.copy()
    for name in cols:
        if name not in df.columns:
            continue
        idxs = np.where(df.columns.values == name)[0]
        if len(idxs) > 1:
            df.drop(columns=df.columns[idxs[1:]], inplace=True)
        df[name] = pd.to_numeric(df[name], errors="coerce")
    return df

def met(y_true, y_pred):
    if len(y_true) == 0 or len(y_pred) == 0:
        return {"note": "no rows"}
    out = {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
    }
    out["r2"] = float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan
    out["n"] = int(len(y_true))
    return out

# load and clean
xl = pd.ExcelFile(EXCEL_PATH)
dfs = []
if "GROUND DATA" in xl.sheet_names:
    g = pd.read_excel(EXCEL_PATH, sheet_name="GROUND DATA")
    g = normalize_headers(g).dropna(how="all")
    g["condition"] = "ground"
    dfs.append(g)

if "FLIGHT DATA" in xl.sheet_names:
    f = pd.read_excel(EXCEL_PATH, sheet_name="FLIGHT DATA")
    f = normalize_headers(f).dropna(how="all")
    f["condition"] = "flight"
    dfs.append(f)

if not dfs:
    raise RuntimeError("Neither 'GROUND DATA' nor 'FLIGHT DATA' sheets found.")

df_all = pd.concat(dfs, ignore_index=True)
df_all = clean_cols(df_all)

# Consolidate duplicates before coercion
cols_for_merge = list(set(EXPECTED_COLS + EXPECTED_FEATURES + [TARGET]))
df_all = consolidate_duplicate_columns(df_all, cols_to_merge=cols_for_merge)

# Parse Material & day + BaseMaterial & DayInt
if "Material and Incubation day" not in df_all.columns:
    raise RuntimeError("Missing 'Material and Incubation day' after header normalization/merge.")
df_all[["Material", "day"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_material_day(s)))
df_all[["BaseMaterial", "DayInt"]] = df_all["Material and Incubation day"].apply(lambda s: pd.Series(parse_base_and_day_v2(s)))

# Coerce numerics
df_all = safe_coerce_numeric(df_all, EXPECTED_FEATURES + [TARGET])

# Keep essentials
df = df_all.dropna(subset=["BaseMaterial", "DayInt", "condition", TARGET]).copy()
df["DayInt"] = pd.to_numeric(df["DayInt"], errors="coerce")
df["condition_bin"] = (df["condition"] == "flight").astype(int)
df["DayInt_x_condition"] = df["DayInt"] * df["condition_bin"]

df.to_csv(OUTDIR / "cleaned_combined.csv", index=False)

existing_feats = [c for c in EXPECTED_FEATURES if c in df.columns]
missing_feats = [c for c in EXPECTED_FEATURES if c not in df.columns]

if len(existing_feats) < len(EXPECTED_FEATURES):
    numeric_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
    fallback_exclude = {TARGET, "day", "DayInt", "condition_bin", "DayInt_x_condition"}
    fallback_feats = [c for c in numeric_candidates if c not in fallback_exclude]
    feature_cols = existing_feats + [c for c in fallback_feats if c not in existing_feats]
else:
    feature_cols = existing_feats

feature_cols = [c for c in feature_cols if df[c].notna().any()]

summary = {
    "rows_total": int(len(df_all)),
    "rows_used": int(len(df)),
    "conditions": df["condition"].value_counts(dropna=False).to_dict(),
    "base_materials": df["BaseMaterial"].value_counts().to_dict(),
    "days": df["DayInt"].value_counts().sort_index().to_dict(),
    "features_used_for_G2F": feature_cols,
    "features_missing_from_expected": missing_feats,
}
with open(OUTDIR / "summary.json", "w") as f:
    json.dump(summary, f, indent=2)

metrics = {}

if ("ground" in df["condition"].unique()) and ("flight" in df["condition"].unique()) and len(feature_cols) > 0:
    df_g = df[(df["condition"] == "ground") & df[TARGET].notna()]
    df_f = df[(df["condition"] == "flight") & df[TARGET].notna()]

    if not df_g.empty and not df_f.empty:
        X_train = df_g[feature_cols].values
        y_train = df_g[TARGET].values
        X_test = df_f[feature_cols].values
        y_test = df_f[TARGET].values

        rf_g2f = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
        rf_g2f.fit(X_train, y_train)
        y_pred_f = rf_g2f.predict(X_test)

        metrics["g2f_regression"] = met(y_test, y_pred_f)
        metrics["g2f_regression"].update({
            "n_train_ground": int(len(df_g)),
            "n_test_flight": int(len(df_f)),
            "n_features": int(len(feature_cols)),
        })

        pred_df = df_f.copy()
        pred_df["predicted_coverage_g2f"] = y_pred_f
        pred_df.to_csv(OUTDIR / "g2f_regression_predictions.csv", index=False)

        # Plot pred vs actual
        plt.figure()
        plt.scatter(y_test, y_pred_f)
        plt.xlabel("Actual Flight Coverage")
        plt.ylabel("Predicted Flight Coverage (trained on Ground)")
        plt.title("Ground→Flight coverage regression (Expert A RF)")
        lo = float(min(y_test.min(), y_pred_f.min()))
        hi = float(max(y_test.max(), y_pred_f.max()))
        plt.plot([lo, hi], [lo, hi], linestyle="--")
        plt.tight_layout()
        plt.savefig(OUTDIR / "g2f_pred_vs_actual.png", dpi=150)
        plt.close()
    else:
        metrics["g2f_regression"] = {"note": "Insufficient ground/flight rows with target."}
else:
    metrics["g2f_regression"] = {"note": "Require both ground and flight and usable features."}

# Numeric features: all numerics except TARGET
num_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
numA = [c for c in num_candidates if c != TARGET]
# Categorical: BaseMaterial and condition
catA = [c for c in ["BaseMaterial", "condition"] if c in df.columns]

df_dom = df[df[TARGET].notna()].copy()

if not df_dom.empty and (len(numA) + len(catA) > 0):
    preA = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), numA),
            ("cat", OneHotEncoder(handle_unknown="ignore"), catA),
        ],
        remainder="drop",
    )
    rf_base = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
    rf_grid = {
        "model__max_depth": [5, 10, None],
        "model__min_samples_leaf": [1, 2, 4],
        "model__max_features": [None, "sqrt"],
    }
    pipeA = Pipeline([("pre", preA), ("model", rf_base)])
    cvA = GridSearchCV(
        pipeA,
        rf_grid,
        cv=KFold(n_splits=3, shuffle=True, random_state=42),
        scoring="r2",
        n_jobs=-1,
    )
    cvA.fit(df_dom[numA + catA], df_dom[TARGET])
    df_dom["pred_expertA"] = cvA.predict(df_dom[numA + catA])
    df_dom.to_csv(OUTDIR / "domain_aware_predictions_expertA.csv", index=False)

    for cond in ["ground", "flight"]:
        sub = df_dom[df_dom["condition"] == cond]
        if not sub.empty:
            m = met(sub[TARGET].values, sub["pred_expertA"].values)
            m["cv_best_params"] = {k: v for k, v in cvA.best_params_.items()}
            metrics[f"domain_model_base_{cond}"] = m
        else:
            metrics[f"domain_model_base_{cond}"] = {"note": "no rows"}
else:
    metrics["domain_model_base"] = {"note": "No rows or no usable features for domain-aware model."}

plots_dir = OUTDIR / "trajectories_base"
plots_dir.mkdir(exist_ok=True)

traj_rows = []
have_flight = "flight" in df["condition"].unique()
materials = sorted(df["BaseMaterial"].dropna().unique())

# Fix a global y-axis range for ALL trajectory plots
Y_MIN, Y_MAX = 0.0, 100.0  # coverage is a percentage; keep it standardized

for mat in materials:
    g_sub = df[(df["condition"] == "ground") & (df["BaseMaterial"] == mat)]
    f_sub = df[(df["condition"] == "flight") & (df["BaseMaterial"] == mat)] if have_flight else pd.DataFrame()

    # Need ≥2 ground days to fit slope
    if g_sub["DayInt"].nunique() < 2:
        continue

    # Linear ground trajectory: coverage ~ DayInt
    Xg = g_sub[["DayInt"]].values
    yg = g_sub[TARGET].values
    lin = LinearRegression().fit(Xg, yg)

    slope = float(lin.coef_[0])
    intercept = float(lin.intercept_)

    # Ground per-day growth rate if Day1 and Day3 exist
    growth_rate = np.nan
    if (g_sub["DayInt"] == 1).any() and (g_sub["DayInt"] == 3).any():
        cov_d1 = g_sub.loc[g_sub["DayInt"] == 1, TARGET].mean()
        cov_d3 = g_sub.loc[g_sub["DayInt"] == 3, TARGET].mean()
        growth_rate = (cov_d3 - cov_d1) / 2.0

    # Method A: ground line evaluated at Day 3
    pred_day3_A = float(lin.predict(np.array([[3]]))[0])

    # Method B: Flight Day1 + ground growth_rate * 2 (if available)
    pred_day3_B = np.nan
    actual_flight_d3 = np.nan
    if have_flight and not f_sub.empty:
        f_d1 = f_sub[f_sub["DayInt"] == 1]
        f_d3 = f_sub[f_sub["DayInt"] == 3]
        if not f_d3.empty:
            actual_flight_d3 = float(f_d3[TARGET].mean())
        if not f_d1.empty and not np.isnan(growth_rate):
            pred_day3_B = float(f_d1[TARGET].mean() + 2.0 * growth_rate)

    traj_rows.append({
        "BaseMaterial": mat,
        "ground_lin_slope": slope,
        "ground_lin_intercept": intercept,
        "ground_growth_rate_per_day": growth_rate,
        "predicted_flight_day3_methodA_ground_line": pred_day3_A,
        "predicted_flight_day3_methodB_flightD1_plus_ground_rate": pred_day3_B,
        "actual_flight_day3": actual_flight_d3,
    })

    plt.figure()
    # Ground points
    plt.scatter(g_sub["DayInt"], g_sub[TARGET], label="Ground (actual)", color="tab:blue")
    # Ground linear fit
    xs = np.array([[1], [2], [3]])
    ys = lin.predict(xs)
    plt.plot(xs.flatten(), ys, linestyle="--", label="Ground fit", color="tab:blue")

    # Flight points + predictions if present
    if have_flight and not f_sub.empty:
        plt.scatter(f_sub["DayInt"], f_sub[TARGET], label="Flight (actual)", color="tab:orange")
        if not np.isnan(pred_day3_B):
            plt.scatter([3], [pred_day3_B], marker="x", label="Pred Flight D3 (Method B)", color="tab:green")
        plt.scatter([3], [pred_day3_A], marker="^", label="Pred Flight D3 (Method A)", color="tab:red")

    plt.xlabel("Day")
    plt.ylabel("Biofilm surface area coverage (%)")
    plt.title(f"Trajectory (BaseMaterial): {mat}")

    # Standardized axes
    plt.xlim(0.8, 3.2)
    plt.xticks([1, 2, 3])
    plt.ylim(Y_MIN, Y_MAX)

    plt.legend()
    plt.tight_layout()
    safe_name = re.sub(r"[^A-Za-z0-9]+", "_", mat).strip("_")
    plt.savefig(plots_dir / f"trajectory_{safe_name}.png", dpi=150)
    plt.close()

traj_df = pd.DataFrame(traj_rows)
traj_df.to_csv(OUTDIR / "trajectory_predictions_base_material.csv", index=False)

def eval_block(df_eval, pred_col):
    if df_eval.empty or pred_col not in df_eval.columns:
        return {"note": "no comparable rows"}
    mask = (~df_eval["actual_flight_day3"].isna()) & (~df_eval[pred_col].isna())
    y_true = df_eval.loc[mask, "actual_flight_day3"].values
    y_pred = df_eval.loc[mask, pred_col].values
    if len(y_true) == 0:
        return {"note": "no comparable rows"}
    return met(y_true, y_pred)

metrics["trajectory_base_methodA"] = eval_block(traj_df, "predicted_flight_day3_methodA_ground_line")
metrics["trajectory_base_methodB"] = eval_block(traj_df, "predicted_flight_day3_methodB_flightD1_plus_ground_rate")

def fit_expertA_domain(train_df):
    """Fit CV-tuned RF (Expert A) on a given training df."""
    df_tr = train_df.copy()
    num_candidates = df_tr.select_dtypes(include=[np.number]).columns.tolist()
    numA_local = [c for c in num_candidates if c != TARGET]
    catA_local = [c for c in ["BaseMaterial", "condition"] if c in df_tr.columns]
    preA_local = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(with_mean=False), numA_local),
            ("cat", OneHotEncoder(handle_unknown="ignore"), catA_local),
        ],
        remainder="drop",
    )
    rf_base_local = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1)
    rf_grid_local = {
        "model__max_depth": [5, 10, None],
        "model__min_samples_leaf": [1, 2, 4],
        "model__max_features": [None, "sqrt"],
    }
    pipeA_local = Pipeline([("pre", preA_local), ("model", rf_base_local)])
    cv_local = GridSearchCV(
        pipeA_local,
        rf_grid_local,
        cv=KFold(n_splits=3, shuffle=True, random_state=42),
        scoring="r2",
        n_jobs=-1,
    )
    X_tr = df_tr[numA_local + catA_local]
    y_tr = df_tr[TARGET].values
    cv_local.fit(X_tr, y_tr)
    return cv_local, numA_local, catA_local

def eval_expertA_domain(model, num_cols, cat_cols, df_eval):
    d = df_eval.copy()
    X = d[num_cols + cat_cols]
    y = d[TARGET].values
    y_pred = model.predict(X)
    return met(y, y_pred), y_pred

# random 70/30 split
train_df, test_df = train_test_split(
    df,
    test_size=0.3,
    random_state=42,
    shuffle=True,
)
model_rand, num_rand, cat_rand = fit_expertA_domain(train_df)
m_train_rand, y_pred_train = eval_expertA_domain(model_rand, num_rand, cat_rand, train_df)
m_test_rand, y_pred_test = eval_expertA_domain(model_rand, num_rand, cat_rand, test_df)

metrics["expertA_randomTrain"] = m_train_rand
metrics["expertA_randomTest"] = m_test_rand

pred_rand = test_df[["Material and Incubation day", "BaseMaterial", "DayInt", "condition", TARGET]].copy()
pred_rand["pred_expertA_blind_random"] = y_pred_test
pred_rand.to_csv(OUTDIR / "expertA_blind_predictions_random_split.csv", index=False)

# 2) Ground→Flight blind test (train on ground only, test on flight)
ground_df = df[df["condition"] == "ground"].copy()
flight_df = df[df["condition"] == "flight"].copy()

if not ground_df.empty and not flight_df.empty:
    model_g2f, num_g2f, cat_g2f = fit_expertA_domain(ground_df)
    m_ground_in, y_pred_ground = eval_expertA_domain(model_g2f, num_g2f, cat_g2f, ground_df)
    m_flight_blind, y_pred_flight = eval_expertA_domain(model_g2f, num_g2f, cat_g2f, flight_df)

    metrics["expertA_G2F_train_ground"] = m_ground_in
    metrics["expertA_G2F_test_flight"] = m_flight_blind

    pred_g2f = flight_df[["Material and Incubation day", "BaseMaterial", "DayInt", "condition", TARGET]].copy()
    pred_g2f["pred_expertA_blind_G2F"] = y_pred_flight
    pred_g2f.to_csv(OUTDIR / "expertA_blind_predictions_G2F.csv", index=False)
else:
    metrics["expertA_G2F"] = {"note": "Need both ground and flight rows for G→F blind test."}

with open(OUTDIR / "metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

with open(OUTDIR / "README.txt", "w") as f:
    f.write(
f"""Outputs: {OUTDIR}
"""
    )


Consolidated duplicate columns: [('Biofilm mass (µm^3/µm^2)', 2)]
Done. Outputs in: /content/lsds55_outputs_expertA


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

OUTDIR = Path("/content/lsds55_outputs_expertA")
TARGET = "Biofilm surface area coverage (%)"
CLEAN_CSV = OUTDIR / "cleaned_combined.csv"

def met(y_true, y_pred):
    if len(y_true) == 0:
        return {"note": "no rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan,
        "n": int(len(y_true)),
    }

df = pd.read_csv(CLEAN_CSV)

# Make sure required columns exist; if not, stop early
required_cols = {"BaseMaterial", "DayInt", "condition", TARGET}
missing = required_cols - set(df.columns)
if missing:
    raise RuntimeError(f"Missing required columns in cleaned_combined.csv: {missing}")

# Drop rows without essential info
df = df.dropna(subset=["BaseMaterial", "DayInt", "condition", TARGET]).copy()
df["condition_bin"] = (df["condition"] == "flight").astype(int)

# Numeric + categorical setup
num_candidates = df.select_dtypes(include=[np.number]).columns.tolist()
if TARGET in num_candidates:
    num_candidates.remove(TARGET)

# Keep DayInt and condition_bin explicitly at the end for readability
num_cols = [c for c in num_candidates if c not in ["DayInt", "condition_bin"]]
for c in ["DayInt", "condition_bin"]:
    if c in df.columns and c not in num_cols:
        num_cols.append(c)

cat_cols = [c for c in ["BaseMaterial", "condition"] if c in df.columns]

pre = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

rf_base = RandomForestRegressor(random_state=42, n_estimators=500, n_jobs=-1)

param_grid = {
    "model__max_depth": [3, 5, None],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": [None],
}

pipe = Pipeline([("pre", pre), ("model", rf_base)])
cv = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
X_all = df[num_cols + cat_cols]
y_all = df[TARGET].values

cv.fit(X_all, y_all)
best_model = cv.best_estimator_
print("Best Expert A params:", cv.best_params_)

# Add Expert A predictions to df
df["expertA_pred"] = best_model.predict(X_all)

plots_dir = OUTDIR / "trajectories_expertA"
plots_dir.mkdir(exist_ok=True)

Y_MIN, Y_MAX = 0.0, 100.0  # standardized axis for coverage
materials = sorted(df["BaseMaterial"].dropna().unique())

traj_rows = []

for mat in materials:
    mat_sub = df[df["BaseMaterial"] == mat].copy()
    g_sub = mat_sub[mat_sub["condition"] == "ground"]
    f_sub = mat_sub[mat_sub["condition"] == "flight"]

    # Need >=2 ground days for linear trajectory
    if g_sub["DayInt"].nunique() < 2:
        continue

    Xg = g_sub[["DayInt"]].values
    yg = g_sub[TARGET].values
    lin = LinearRegression().fit(Xg, yg)

    slope = float(lin.coef_[0])
    intercept = float(lin.intercept_)

    # Ground growth rate (Day1 -> Day3) for Method B
    growth_rate = np.nan
    if (g_sub["DayInt"] == 1).any() and (g_sub["DayInt"] == 3).any():
        cov_d1 = g_sub.loc[g_sub["DayInt"] == 1, TARGET].mean()
        cov_d3 = g_sub.loc[g_sub["DayInt"] == 3, TARGET].mean()
        growth_rate = (cov_d3 - cov_d1) / 2.0

    pred_day3_A = float(lin.predict(np.array([[3]]))[0])

    pred_day3_B = np.nan
    actual_flight_d3 = np.nan
    if not f_sub.empty:
        f_d1 = f_sub[f_sub["DayInt"] == 1]
        f_d3 = f_sub[f_sub["DayInt"] == 3]
        if not f_d3.empty:
            actual_flight_d3 = float(f_d3[TARGET].mean())
        if not f_d1.empty and not np.isnan(growth_rate):
            pred_day3_B = float(f_d1[TARGET].mean() + 2.0 * growth_rate)

    traj_rows.append({
        "BaseMaterial": mat,
        "ground_lin_slope": slope,
        "ground_lin_intercept": intercept,
        "ground_growth_rate_per_day": growth_rate,
        "predicted_flight_day3_methodA_ground_line": pred_day3_A,
        "predicted_flight_day3_methodB_flightD1_plus_ground_rate": pred_day3_B,
        "actual_flight_day3": actual_flight_d3,
    })

    # Ground
    if not g_sub.empty:
        g_mean = (
            g_sub.groupby("DayInt")
            .agg(actual=(TARGET, "mean"), pred=("expertA_pred", "mean"))
            .reset_index()
        )
    else:
        g_mean = None

    # Flight
    if not f_sub.empty:
        f_mean = (
            f_sub.groupby("DayInt")
            .agg(actual=(TARGET, "mean"), pred=("expertA_pred", "mean"))
            .reset_index()
        )
    else:
        f_mean = None

    plt.figure()

    # Ground actual scatter
    plt.scatter(g_sub["DayInt"], g_sub[TARGET],
                label="Ground (actual)", color="tab:blue")

    # Ground linear fit (old "Ground fit")
    xs = np.array([[1], [2], [3]])
    ys = lin.predict(xs)
    plt.plot(xs.flatten(), ys, linestyle="--",
             label="Ground fit (linear)", color="tab:blue")

    # Flight actual scatter
    if not f_sub.empty:
        plt.scatter(f_sub["DayInt"], f_sub[TARGET],
                    label="Flight (actual)", color="tab:orange")

    # Old trajectory predictions (Method A & B)
    if not np.isnan(pred_day3_B):
        plt.scatter([3], [pred_day3_B], marker="x",
                    label="Pred Flight D3 (Method B)", color="tab:green")
    plt.scatter([3], [pred_day3_A], marker="^",
                label="Pred Flight D3 (Method A)", color="tab:red")

    # Expert A ground predictions (day-wise mean)
    if g_mean is not None and len(g_mean) > 0:
        plt.plot(
            g_mean["DayInt"],
            g_mean["pred"],
            "-.",
            color="tab:purple",
            label="Ground Expert A (mean pred)",
        )

    # Expert A flight predictions (day-wise mean)
    if f_mean is not None and len(f_mean) > 0:
        plt.plot(
            f_mean["DayInt"],
            f_mean["pred"],
            "-.",
            color="tab:brown",
            label="Flight Expert A (mean pred)",
        )

    plt.xlabel("Day")
    plt.ylabel("Biofilm surface area coverage (%)")
    plt.title(f"Trajectory with Expert A (BaseMaterial): {mat}")
    plt.xlim(0.8, 3.2)
    plt.xticks([1, 2, 3])
    plt.ylim(Y_MIN, Y_MAX)
    plt.legend()
    plt.tight_layout()

    safe_name = "".join(ch if ch.isalnum() else "_" for ch in mat).strip("_")
    plt.savefig(plots_dir / f"trajectory_expertA_{safe_name}.png", dpi=150)
    plt.close()

traj_df = pd.DataFrame(traj_rows)
traj_df.to_csv(plots_dir / "trajectory_expertA_summary.csv", index=False)

print(f"Done. Expert A trajectory plots saved in: {plots_dir}")


Best Expert A params: {'model__max_depth': 3, 'model__max_features': None, 'model__min_samples_leaf': 2}
Done. Expert A trajectory plots saved in: /content/lsds55_outputs_expertA/trajectories_expertA


## NEW Code added below, aligning with Method A of the Current Results in the project design paper

In [ ]:
# -*- coding: utf-8 -*-
"""
This script is a little different from the above ones in that it trains on ground-flight data and tests using only ground inputs instead of flight inputs.
"""

from pathlib import Path
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


CSV_PATH = Path("cleaned_combined_updated.csv")   
OUTDIR = Path("lsds55_outputs_strict_g2f") 
OUTDIR.mkdir(parents=True, exist_ok=True)

TARGET = "Biofilm surface area coverage (%)"  # flight y-label
COL_MAX_THICK = "Biofilm Maximum thickness (µm)"
COL_ROUGH_BIO = "roughness coefficient Ra*"
COL_MAT_ROUGH = "Material_Roughness"
COL_CA = "Contact_Angle"
COL_COSTH = "Cos(theta)"
COL_WADH = "Wadh"


def met(y_true, y_pred):
    """Compute RMSE/MAE/R2 + n."""
    if len(y_true) == 0:
        return {"note": "no rows"}
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)) if len(np.unique(y_true)) > 1 else np.nan,
        "n": int(len(y_true)),
    }


if not CSV_PATH.exists():
    raise FileNotFoundError(f"Cannot find {CSV_PATH}. Make sure cleaned_combined.csv is next to this script.")

df = pd.read_csv(CSV_PATH)

required_cols = [
    "BaseMaterial",
    "DayInt",
    "condition",
    TARGET,
    COL_MAX_THICK,
    COL_ROUGH_BIO,
    COL_MAT_ROUGH,
    COL_CA,
    COL_COSTH,
    COL_WADH,
]

missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise RuntimeError(f"Missing required columns in cleaned_combined.csv: {missing}")

df["DayInt"] = pd.to_numeric(df["DayInt"], errors="coerce")
df = df.dropna(subset=["BaseMaterial", "DayInt", "condition", TARGET]).copy()


# Separate ground and flight
df_ground = df[df["condition"].str.lower() == "ground"].copy()
df_flight = df[df["condition"].str.lower() == "flight"].copy()

group_cols = ["BaseMaterial", "DayInt"]

g_groups = df_ground.groupby(group_cols)
f_groups = df_flight.groupby(group_cols)

common_keys = sorted(set(g_groups.groups.keys()) & set(f_groups.groups.keys()))

paired_rows = []

for (mat, day) in common_keys:
    g_block = g_groups.get_group((mat, day))
    f_block = f_groups.get_group((mat, day))

    # Aggregate GROUND-side inputs (mean over replicates)
    row = {
        "BaseMaterial": mat,
        "DayInt": float(day),
        "ground_coverage_mean": float(g_block[TARGET].mean()),
        "ground_max_thickness_mean": float(g_block[COL_MAX_THICK].mean()),
        "ground_roughness_bio_mean": float(g_block[COL_ROUGH_BIO].mean()),
        # Material properties (should be constant per material, but we average just in case):
        "Material_Roughness": float(g_block[COL_MAT_ROUGH].mean()),
        "Contact_Angle": float(g_block[COL_CA].mean()),
        "Cos_theta": float(g_block[COL_COSTH].mean()),
        "Wadh": float(g_block[COL_WADH].mean()),
        # FLIGHT coverage = target y (mean)
        "flight_coverage_mean": float(f_block[TARGET].mean()),
    }

    paired_rows.append(row)

paired_df = pd.DataFrame(paired_rows)

if paired_df.empty:
    raise RuntimeError("No (BaseMaterial, DayInt) pairs with both ground and flight data found.")


num_feature_cols = [
    "DayInt",
    "ground_coverage_mean",
    "ground_max_thickness_mean",
    "ground_roughness_bio_mean",
    "Material_Roughness",
    "Contact_Angle",
    "Cos_theta",
    "Wadh",
]
cat_feature_cols = ["BaseMaterial"]

for c in num_feature_cols:
    paired_df[c] = pd.to_numeric(paired_df[c], errors="coerce")

paired_df = paired_df.dropna(subset=num_feature_cols + ["flight_coverage_mean"]).copy()

X = paired_df[num_feature_cols + cat_feature_cols]
y = paired_df["flight_coverage_mean"].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, shuffle=True
)

# Preprocessing: scale numerics, one-hot encode BaseMaterial
pre = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), num_feature_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_feature_cols),
    ],
    remainder="drop",
)

rf = RandomForestRegressor(
    n_estimators=600,
    random_state=42,
    n_jobs=-1,
)

pipe = Pipeline([
    ("pre", pre),
    ("rf", rf),
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

# post-hoc linear calibration to fix scaling issue
from sklearn.linear_model import LinearRegression

# reshape for sklearn
X_cal = y_pred.reshape(-1, 1)
y_cal = y_test

cal = LinearRegression().fit(X_cal, y_cal)
a = cal.intercept_
b = cal.coef_[0]

print("\nCalibration fitted: y_true ≈ a + b * y_pred")
print("a =", a, "   b =", b)

# apply calibration
y_pred_cal = a + b * y_pred

# physical clipping to avoid negative or >100 values
y_pred_cal = np.clip(y_pred_cal, 0.0, 100.0)


metrics = {}
metrics["strict_ground_to_flight"] = met(y_test, y_pred_cal)
metrics["strict_ground_to_flight"].update({
    "calibration_a": float(a),
    "calibration_b": float(b),
})
metrics["strict_ground_to_flight"].update({
    "n_pairs_total": int(len(paired_df)),
    "n_train_pairs": int(len(X_train)),
    "n_test_pairs": int(len(X_test)),
    "numeric_features": num_feature_cols,
    "categorical_features": cat_feature_cols,
})

with open(OUTDIR / "metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

# Save predictions (test set only)
pred_df = X_test.copy()
pred_df["y_true_flight"] = y_test
pred_df["y_pred_flight_calibrated"] = y_pred_cal
pred_df["y_pred_flight_raw"] = y_pred  # (optional, for debugging)

pred_df.to_csv(OUTDIR / "strict_ground_to_flight_predictions.csv", index=False)

# Pred vs actual scatter
plt.figure()
plt.scatter(y_test, y_pred_cal)
plt.xlabel("Actual Flight Coverage (mean)")
plt.ylabel("Calibrated Predicted Flight Coverage")
plt.title("Strict Ground→Flight Regression (after calibration)")

lo = float(min(y_test.min(), y_pred_cal.min()))
hi = float(max(y_test.max(), y_pred_cal.max()))
plt.plot([lo, hi], [lo, hi], linestyle="--")

plt.tight_layout()
plt.savefig(OUTDIR / "strict_ground_to_flight_pred_vs_actual_CALIBRATED.png", dpi=150)

plt.close()

print("Done.")
print("Pairs total:", len(paired_df))
print("Train pairs:", len(X_train), " Test pairs:", len(X_test))
print("Metrics written to:", OUTDIR / "metrics.json")
print("Predictions CSV:", OUTDIR / "strict_ground_to_flight_predictions.csv")
print("Pred vs Actual plot:", OUTDIR / "strict_ground_to_flight_pred_vs_actual.png")




Calibration fitted: y_true ≈ a + b * y_pred
a = -20.43978786518256    b = 2.2410182636970144
Done.
Pairs total: 18
Train pairs: 12  Test pairs: 6
Metrics written to: lsds55_outputs_strict_g2f\metrics.json
Predictions CSV: lsds55_outputs_strict_g2f\strict_ground_to_flight_predictions.csv
Pred vs Actual plot: lsds55_outputs_strict_g2f\strict_ground_to_flight_pred_vs_actual.png
